<a href="https://colab.research.google.com/github/clarafou/ECTN/blob/main/Terminaux_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install GitPython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.4 MB/s eta 0:00:00


In [8]:
# Importez les bibliothèques nécessaires
import os
import shutil
import git

# Spécifiez votre nom d'utilisateur GitHub et votre adresse e-mail
git_username = "clarafou"
git_email = "clara.fourrier@gmail.com"

# Configurez Git avec vos informations
git.Git().config("--global", "user.name", git_username)
git.Git().config("--global", "user.email", git_email)

# Définissez l'URL de votre référentiel GitHub
repo_url = "https://github.com/clarafou/ECTN.git"

# Vérifiez si le référentiel existe déjà, sinon, clonez-le
if not os.path.exists("ECTN"):
    git.Repo.clone_from(repo_url, "ECTN")

# Changez le répertoire de travail vers votre référentiel cloné
os.chdir("ECTN")



In [9]:
!git branch


* main


In [11]:
!git add Terminaux_Final  # Remplacez "nom_du_fichier" par le nom réel de votre fichier
!git commit -m "Ajout du fichier depuis Google Colab"
!git push origin master


fatal: pathspec 'Terminaux_Final' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/clarafou/ECTN.git'


In [ ]:
import subprocess
import sys
import os
import re
import atexit
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import datetime as dt
import pickle


# Data Loading


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
path = "drive/MyDrive/ECTN_Routing/"
os.listdir(path)

['02_NUTS-3-Regions.csv',
 '03_network-nodes.csv',
 '04_network-edges.csv',
 'Corridor_97%.xlsx',
 'Eligibility_Type_Goods.xlsx',
 'Terminaux.xlsx',
 'Terminaux_Coor.xlsx',
 'distance_matrix_haversine.xlsx',
 'distance_matrix.xlsx',
 'Copie de Terminaux_Final.ipynb',
 'distance_matrix_matricelike.xlsx',
 'Terminals_to_Fill Modif NTH_BIS.xlsx',
 'distance_matrix_bis.xlsx',
 'Terminaux_Data_Stocké.xlsx',
 'Nodes_Data_Stocké.xlsx',
 'distance_matrix_TOT.xlsx',
 'NUTS3_Stocké.xlsx',
 'Flux_Stocké.xlsx',
 'Flux_Scaled_Stocké.xlsx',
 'Nodes_Stocké.xlsx',
 'Terminaux_Stocké.xlsx',
 'ECTN_Terminals_INPUT (4).xlsx',
 'Find_Nodes.ipynb',
 'distance_matrix_3.xlsx',
 'Distance_Matrix.ipynb',
 'Terminaux_Noded.xlsx',
 'Flux.xlsx',
 'Terminaux_Final.ipynb']

In [ ]:
Terminaux_Data = pd.read_excel("drive/MyDrive/ECTN_Routing/Terminaux_Noded.xlsx" , header=0)
NUTS3_Data = pd.read_csv("drive/MyDrive/ECTN_Routing/02_NUTS-3-Regions.csv")
Nodes_Data = pd.read_csv("drive/MyDrive/ECTN_Routing/03_network-nodes.csv")
Edges_Data= pd.read_csv("drive/MyDrive/ECTN_Routing/04_network-edges.csv")

# Data Preparation

Nous avons des Terminaux en INPUT, que l'on a placé de manière pertinente en précisant leur date de lancement et leur localisation et en leur affectant le noeud du réseau autoroutier ETISplus le plus proche.

In [ ]:
Terminaux_Data

,Terminal_Name,Terminal_Lat,Terminal_Long,Step,Go Live Planned,Id_Node,Node_Lat,Node_Long,Transit
0,Aarhus,56.149628,10.213405,S3_Ph2,2036,116479,56.135541,10.156370,N
1,Albacete - San Clemente,39.403889,-2.429444,S1_Ph3,2030,250086,39.351198,-2.441125,Y
2,Algésiras,36.127500,-5.453889,S1_Ph3,2030,101665,36.112912,-5.455857,N
3,Alicante - Crevillent,38.248611,-0.808889,S1_Ph3,2030,101570,38.220704,-0.812785,Y
4,Almería,36.840000,-2.467500,S1_Ph3,2030,101645,36.838037,-2.447375,N
...,...,...,...,...,...,...,...,...,...
175,Wuppertal,51.266667,7.183333,S1_Ph2,2027,196437,51.279481,7.179971,"['Cologne', ' Moers', ' Maastricht', ' Eindhov..."
176,Wurtzbourg,49.794444,9.929444,S1_Ph2,2027,108180,49.753068,9.927837,"['Nuremberg - Fürth','Munich']"
177,Zagreb,45.842641,15.962231,S2_Ph2,2033,103613,45.753262,15.989232,Y
178,Zielona Góra,51.939722,15.505000,S1_Ph2,2027,198328,51.924905,15.561454,Y


In [ ]:
#Je trouve le pays associé à mon terminal
Terminaux_Data = Terminaux_Data.merge(Nodes_Data[['Network_Node_ID', 'Country']], left_on='Id_Node', right_on='Network_Node_ID')
Terminaux_Data = Terminaux_Data.drop(columns=['Id_Node'])

In [ ]:
#On vérifie le nombre de terminaux
nombre_de_lignes = Terminaux_Data.shape[0]
print(nombre_de_lignes)

180


In [ ]:
#On vérifie que tous les Latitues/Longitudes sont valides
nan_values = Terminaux_Data['Node_Lat'].isna().any()

# Pour vérifier les valeurs uniques dans la colonne "latitude"
unique_values = Terminaux_Data['Node_Lat'].unique()

if nan_values:
    print("Il y a des valeurs NaN dans la colonne 'latitude'")
else:
    print("Il n'y a pas de valeurs NaN dans la colonne 'latitude'")

# Si vous avez une plage valide de valeurs de latitude, par exemple entre -90 et 90 degrés
valid_latitude_range = (-90, 90)

# Filtrer les valeurs en dehors de la plage valide
invalid_values = Terminaux_Data[(Terminaux_Data['Node_Lat'] < valid_latitude_range[0]) | (Terminaux_Data['Node_Lat'] > valid_latitude_range[1])]

if not invalid_values.empty:
    print("Les valeurs suivantes sont en dehors de la plage valide pour la latitude:")
    print(invalid_values)
else:
    print("Toutes les valeurs de latitude sont valides.")



Il n'y a pas de valeurs NaN dans la colonne 'latitude'
Toutes les valeurs de latitude sont valides.


## Définition périmètre

On définit un périmètre d'aggrégation : Pour tous les temrminaux en INPUT on va venir capter tous les flux à moins de 100km.

In [ ]:
pip install haversine

In [ ]:
from haversine import haversine, Unit

In [ ]:
def point_dans_rayon(waypoint_term,waypoint_node, rayon_km=100):
    # Calcul de la distance entre les deux points
    distance = haversine(waypoint_term,waypoint_node,unit=Unit.KILOMETERS)

    # Vérification si la distance est inférieure ou égale au rayon en kilomètres
    if distance <= rayon_km:
        return True
    else:
        return False

## Création List_Nodes_Hub

On créé pour chaque terminaux une colonne List_Nodes_Hub qui sont une liste de tous les IDs des noeuds présents dans ce périmètre. Si un noeud apparait dans le périmètre de deux terminaux à la fois, on l'affecte au terminal le plus proche.

In [ ]:
Terminaux_Data['List_Nodes_Hub'] = Terminaux_Data.apply(
    lambda row: Nodes_Data.loc[
        Nodes_Data.apply(lambda node_row: point_dans_rayon((row['Node_Lat'], row['Node_Long']), (node_row['Network_Node_Y'], node_row['Network_Node_X'])), axis=1),
        'Network_Node_ID'
    ].tolist(),
    axis=1
)

# Afficher le DataFrame avec la colonne List_Nodes mise à jour
Terminaux_Data.head()

,Terminal_Name,Terminal_Lat,Terminal_Long,Step,Go Live Planned,Node_Lat,Node_Long,Transit,Network_Node_ID,Country,List_Nodes_Hub
0,Aarhus,56.149628,10.213405,S3_Ph2,2036,56.135541,10.156370,N,116479,DK,"[262458, 250470, 116447, 130106, 130103, 12437..."
1,Albacete - San Clemente,39.403889,-2.429444,S1_Ph3,2030,39.351198,-2.441125,Y,250086,ES,"[193560, 193559, 250086, 191616, 101311, 10132..."
2,Algésiras,36.127500,-5.453889,S1_Ph3,2030,36.112912,-5.455857,N,101665,ES,"[101772, 101658, 101659, 101766, 124426, 10176..."
3,Alicante - Crevillent,38.248611,-0.808889,S1_Ph3,2030,38.220704,-0.812785,Y,101570,ES,"[193423, 101815, 101555, 101564, 101814, 10182..."
4,Almería,36.840000,-2.467500,S1_Ph3,2030,36.838037,-2.447375,N,101645,ES,"[250093, 101653, 101705, 101708, 101656, 10170..."


In [ ]:
#On veut checker les noeuds qui apparaissent dans la liste de plusieurs terminaux
from collections import defaultdict

# Créez un dictionnaire pour stocker les terminaux associés à chaque ID de nœud
node_id_to_terminals = defaultdict(list)

# Parcourez chaque ligne de Terminaux_Data
for index, row in Terminaux_Data.iterrows():
    terminal_name = row['Terminal_Name']
    node_ids = row['List_Nodes_Hub']

    # Parcourez les IDs de nœuds dans la liste
    for node_id in node_ids:
        node_id_to_terminals[node_id].append(terminal_name)

# Recherchez les ID de nœuds qui apparaissent dans plusieurs terminaux
duplicate_node_terminals = {node_id: terminals for node_id, terminals in node_id_to_terminals.items() if len(terminals) > 1}

if len(duplicate_node_terminals) > 0:
    print("Les IDs de nœuds suivants apparaissent dans plusieurs terminaux :")
    for node_id, terminals in duplicate_node_terminals.items():
        terminals_str = ', '.join(str(t) for t in terminals)
        print(f"{node_id} : {terminals_str}")
else:
    print("Aucun ID de nœud n'apparaît dans plusieurs terminaux.")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
115214 : Brême, Hambourg, Hanovre
197100 : Brême, Hambourg
116051 : Brême, Hambourg
197099 : Brême, Hambourg
115258 : Brême, Hambourg
197095 : Brême, Hambourg
115253 : Brême, Hambourg
197097 : Brême, Hambourg
197096 : Brême, Hambourg
197101 : Brême, Hambourg
197103 : Brême, Hambourg
115211 : Brême, Hambourg
115212 : Brême, Hambourg
261148 : Brême, Emmen
115248 : Brême, Hambourg
197093 : Brême, Hambourg
197091 : Brême, Hambourg
197088 : Brême, Hambourg
186310 : Brême, Emmen
115675 : Brême, Emmen
196719 : Brême, Emmen
197072 : Brême, Hambourg, Hanovre
197069 : Brême, Hambourg, Hanovre
197055 : Brême, Hambourg, Hanovre
197054 : Brême, Hambourg, Hanovre
197070 : Brême, Hambourg
115203 : Brême, Hambourg
197073 : Brême, Hambourg
196706 : Brême, Emmen
196699 : Brême, Emmen
196676 : Brême, Emmen
112376 : Brême, Emmen
196682 : Brême, Emmen
115257 : Brême, Hambourg
115199 : Brême, Hambourg
115219 : Brême, Hambourg
1156

## Affectation Nodes au terminal le plus proche

Ici on affecte

In [ ]:
#On veut affecter à chaque noeud le terminal le plus proche
import numpy as np

# Fonction pour obtenir le terminal le plus proche en utilisant List_Nodes_Hub
def get_nearest_terminal(row, df_terminaux):
    node_x = row['Network_Node_X']
    node_y = row['Network_Node_Y']

    min_distance = float('inf')  # Initialiser la distance minimale à l'infini
    closest_terminal_name = "Not Scaled"
    closest_terminal_id = "Not Scaled"

    for _, terminal_row in df_terminaux.iterrows():
        terminal_name = terminal_row['Terminal_Name']
        terminal_x = terminal_row['Node_Long']
        terminal_y = terminal_row['Node_Lat']

        # Vérifier si le node_id du nœud se trouve dans List_Nodes_Hub du terminal
        node_id = row['Network_Node_ID']
        hub_list = terminal_row['List_Nodes_Hub']
        if node_id in hub_list:
            distance = haversine((terminal_y,terminal_x),(node_y,node_x))

            if distance < min_distance:
                min_distance = distance
                closest_terminal_name = f"Hub_{terminal_name}({terminal_row['Country']})"
                closest_terminal_id = terminal_row['Network_Node_ID']

    return pd.Series({'Name_Scaled': closest_terminal_name, 'ID_Node_Scaled': closest_terminal_id})

# Appliquer la fonction pour créer les colonnes 'Name_Scaled' et 'ID_Node_Scaled' dans 'Nodes_Data'
Nodes_Data[['Name_Scaled', 'ID_Node_Scaled']] = Nodes_Data.apply(get_nearest_terminal, args=(Terminaux_Data,), axis=1)


In [ ]:
import pandas as pd

Nodes_Data['Name_City'] = Nodes_Data['Name_Scaled'].str.extract(r'Hub_(.*?)\([^)]+\)')

# Cela extraira le nom de la ville entre 'Hub_' et la première paire de parenthèses
# indépendamment de ce qui se trouve entre les parenthèses


In [ ]:
# Compter le nombre de lignes où 'Name_Scaled' n'est pas égal à "Not Scaled"
nombre_lignes_scaled = len(Nodes_Data[Nodes_Data['Name_Scaled'] != "Not Scaled"])

print(f"Nombre de noeuds avec un nom 'Scaled': {nombre_lignes_scaled}")

Nombre de noeuds avec un nom 'Scaled': 12694


In [ ]:
#On vérifie que ce chiffre est bien inférieur ou egal à la somme des longueurs des liste de chaque terminaux
somme_longueur_listes = Terminaux_Data['List_Nodes_Hub'].apply(len).sum()

print(f"Somme des longueurs des listes dans la colonne 'List_Nodes_Hub' : {somme_longueur_listes}")


Somme des longueurs des listes dans la colonne 'List_Nodes_Hub' : 26082


In [ ]:
# Créez un dictionnaire de correspondance entre les anciens et les nouveaux noms de colonnes
column_mapping = {
    'Network_Node_ID': 'ID_Node_Original',
    'Network_Node_X': 'Longitude_Original',
    'Network_Node_Y': 'Latitude_Original',
    'ETISplus_Zone_ID': 'ID_NUTS3_Original',
    'Country': 'Country_Original'
}

# Utilisez la méthode .rename() pour renommer les colonnes
Nodes_Data = Nodes_Data.rename(columns=column_mapping)


In [ ]:
Nodes_Data_Original = pd.read_csv("drive/MyDrive/ECTN_Routing/03_network-nodes.csv")

In [ ]:
# Ensuite, vous pouvez essayer de fusionner les DataFrames
Nodes_Data = Nodes_Data.merge(Nodes_Data_Original[['Network_Node_ID', 'Network_Node_X', 'Network_Node_Y',
                                                   'ETISplus_Zone_ID', 'Country']], left_on='ID_Node_Scaled',
                               right_on='Network_Node_ID')

# Renommez les colonnes
Nodes_Data = Nodes_Data.rename(columns={
    'Network_Node_X': 'Longitude_Scaled',
    'Network_Node_Y': 'Latitude_Scaled',
    'ETISplus_Zone_ID': 'ID_NUTS3_Scaled',
    'Country': 'Country_Scaled'
})


# Supprimez la colonne 'Network_Node_ID'
Nodes_Data = Nodes_Data.drop(columns='Network_Node_ID')

In [ ]:
num_rows = Nodes_Data.shape[0]
print(f"Le nombre de lignes dans Nodes_Data est : {num_rows}")

Le nombre de lignes dans Nodes_Data est : 12694


In [ ]:
num_rows = Nodes_Data_Original.shape[0]
print(f"Le nombre de lignes dans Nodes_Data_Original est : {num_rows}")

Le nombre de lignes dans Nodes_Data_Original est : 17435


In [ ]:
# Compter le nombre de lignes où 'Name_Scaled' n'est pas égal à "Not Scaled"
nombre_lignes_scaled = len(Nodes_Data[Nodes_Data['Name_Scaled'] != "Not Scaled"])

print(f"Nombre de noeuds avec un nom 'Scaled': {nombre_lignes_scaled}")

Nombre de noeuds avec un nom 'Scaled': 12694


In [ ]:
#On veut créer une colonne qui indique la distance entre le noeud et le terminal associé
import pandas as pd
from haversine import haversine

def calculate_haversine(row):
    orig_lat = row['Latitude_Original']
    orig_long = row['Longitude_Original']
    scaled_lat = row['Latitude_Scaled']
    scaled_long = row['Longitude_Scaled']
    return haversine((orig_lat, orig_long), (scaled_lat, scaled_long))

# Appliquez la fonction pour calculer la distance haversine et créez la colonne 'Distance_Term'
Nodes_Data['Distance_Term'] = Nodes_Data.apply(calculate_haversine, axis=1)

# Affichage des premières lignes du DataFrame avec la colonne 'Distance_Term' ajoutée
Nodes_Data.head()


,Unnamed: 0,ID_Node_Original,Longitude_Original,Latitude_Original,ID_NUTS3_Original,Country_Original,Name_Scaled,ID_Node_Scaled,Name_City,Longitude_Scaled,Latitude_Scaled,ID_NUTS3_Scaled,Country_Scaled,Distance_Term
0,0,102872,2.862483,42.695843,112080105,FR,Hub_Perpignan - Pollestres(FR),260366,Perpignan - Pollestres,2.852639,42.608395,112080105,FR,9.757055
1,1,102340,2.879551,42.729368,112080105,FR,Hub_Perpignan - Pollestres(FR),260366,Perpignan - Pollestres,2.852639,42.608395,112080105,FR,13.630368
2,122,102874,2.862069,42.671508,112080105,FR,Hub_Perpignan - Pollestres(FR),260366,Perpignan - Pollestres,2.852639,42.608395,112080105,FR,7.060117
3,155,260369,3.127974,43.232866,112080101,FR,Hub_Perpignan - Pollestres(FR),260366,Perpignan - Pollestres,2.852639,42.608395,112080105,FR,72.967658
4,256,102775,2.983430,43.149954,112080101,FR,Hub_Perpignan - Pollestres(FR),260366,Perpignan - Pollestres,2.852639,42.608395,112080105,FR,61.154427


In [ ]:
def verifie_distances(dataframe):
    # Vérifie si la colonne 'Distance_Term' contient une distance supérieure à 100 km
    max_distance = dataframe['Distance_Term'].max()
    if max_distance <= 100:
        return True
    else:
        return False

In [ ]:
resultat_verification = verifie_distances(Nodes_Data)

if resultat_verification:
    print("Toutes les distances sont inférieures ou égales à 100 km.")
else:
    print("Au moins une distance est supérieure à 100 km.")


Toutes les distances sont inférieures ou égales à 100 km.


In [ ]:
#On merge NUTS3 et Nodes sur le Node_Id et on développe Name_scaled et Id_Scaled
NUTS3_Data = NUTS3_Data.merge(Nodes_Data[['ID_Node_Original','Name_City','Name_Scaled','ID_NUTS3_Scaled','ID_Node_Scaled']], left_on='Network_Node_ID',right_on = 'ID_Node_Original')
# Renommer une colonne
NUTS3_Data = NUTS3_Data.rename(columns={'ETISPlus_Zone_ID': 'ID_Nuts3_original'})
NUTS3_Data = NUTS3_Data.rename(columns={'Name': 'Name_Nuts3_Original'})
NUTS3_Data = NUTS3_Data.rename(columns={'Name_Scaled': 'Name_Nuts3_Scaled'})
NUTS3_Data = NUTS3_Data.rename(columns={'Name_City': 'Name_City_Scaled'})
NUTS3_Data = NUTS3_Data.rename(columns={'Geometric_center_X': 'Geometric_center_X_Original'})
NUTS3_Data = NUTS3_Data.rename(columns={'Geometric_center_Y': 'Geometric_center_Y_Original'})

In [ ]:
NUTS3_Data = NUTS3_Data.drop(columns=['Network_Node_ID'])

In [ ]:
ligne_corridor = NUTS3_Data.loc[NUTS3_Data['Name_Nuts3_Original'] == 'Moselle']
ligne_corridor

,Unnamed: 0,ID_Nuts3_original,Name_Nuts3_Original,Country,Geometric_center,Geometric_center_X_Original,Geometric_center_Y_Original,ID_Node_Original,Name_City_Scaled,Name_Nuts3_Scaled,ID_NUTS3_Scaled,ID_Node_Scaled
262,447,112040103,Moselle,FR,POINT (6.76657 49.0195),6.76657,49.0195,108760,Sarrebruck,Hub_Sarrebruck(DE),107120001,261185


In [ ]:
num_rows = NUTS3_Data.shape[0]
print(f"Le nombre de lignes dans NUTS3_Data est : {num_rows}")

Le nombre de lignes dans NUTS3_Data est : 1123


In [ ]:
NUTS3_Data_Original = pd.read_csv("drive/MyDrive/ECTN_Routing/02_NUTS-3-Regions.csv")

In [ ]:
num_rows = NUTS3_Data_Original.shape[0]
print(f"Le nombre de lignes dans NUTS3_Data_Original est : {num_rows}")

Le nombre de lignes dans NUTS3_Data_Original est : 1675


# Import de la table Flux

In [ ]:
Flux_Data = pd.read_excel("drive/MyDrive/ECTN_Routing/Corridor_97%.xlsx", sheet_name=2, header=0)

In [ ]:
nombre_de_lignes = Flux_Data.shape[0]
print(f"Le nombre de lignes dans Flux_Data est : {nombre_de_lignes}")

Le nombre de lignes dans Flux_Data est : 76799


In [ ]:
#On ne garde que les flux qui sont associés à un terminal
Flux_Data = Flux_Data.merge(NUTS3_Data[['ID_Nuts3_original','ID_Node_Original','Name_Nuts3_Scaled','ID_Node_Scaled','ID_NUTS3_Scaled']], left_on='ID_origin_NUTS3',right_on = 'ID_Nuts3_original')
#On créé les colonnes Name_Scaled_Origin
Flux_Data = Flux_Data.rename(columns={'Name_Nuts3_Scaled': 'Name_Scaled_Origin'})
Flux_Data = Flux_Data.rename(columns={'ID_Node_Scaled': 'ID_Node_Scaled_Origin'})
Flux_Data = Flux_Data.rename(columns={'ID_NUTS3_Scaled': 'ID_NUTS3_Scaled_Origin'})
Flux_Data = Flux_Data.rename(columns={'ID_Node_Original': 'ID_Node_Original_Origin'})

In [ ]:
#On créé les colonnes Name_Scaled_Destination
Flux_Data = Flux_Data.merge(NUTS3_Data[['ID_Nuts3_original','ID_Node_Original','Name_Nuts3_Scaled','ID_Node_Scaled','ID_NUTS3_Scaled']], left_on='ID_destination_NUTS3',right_on = 'ID_Nuts3_original')
Flux_Data = Flux_Data.rename(columns={'Name_Nuts3_Scaled': 'Name_Scaled_Destination'})
Flux_Data = Flux_Data.rename(columns={'ID_Node_Scaled': 'ID_Node_Scaled_Destination'})
Flux_Data = Flux_Data.rename(columns={'ID_NUTS3_Scaled': 'ID_NUTS3_Scaled_Destination'})
Flux_Data = Flux_Data.rename(columns={'ID_Node_Original': 'ID_Node_Original_Destination'})

In [ ]:
#On supprime les colonnes en trop.
Flux_Data = Flux_Data.drop(columns=['ID_Nuts3_original_x'])
Flux_Data = Flux_Data.drop(columns=['ID_Nuts3_original_y'])

In [ ]:
nombre_de_lignes = Flux_Data.shape[0]
print(f"Le nombre de lignes dans Flux_Data est : {nombre_de_lignes}")

Le nombre de lignes dans Flux_Data est : 65970


In [ ]:
Flux_Data.columns

Index(['ID_origin_NUTS3', 'Name_origin_NUTS3', 'ID_destination_NUTS3',
       'Name_destination_NUTS3', 'Edge_path_E_road',
       'Distance_from_origin_region_to_E_road', 'Distance_within_E_road',
       'Distance_from_E_road_to_destination_region', 'Total_distance_Mendeley',
       'Traffic_flow_trucks_2019', 'Traffic_flow_tons_2019',
       'Corridor_générique', 'Total_Distance_PTV', 'Eligibilité ',
       'ID_Node_Original_Origin', 'Name_Scaled_Origin',
       'ID_Node_Scaled_Origin', 'ID_NUTS3_Scaled_Origin',
       'ID_Node_Original_Destination', 'Name_Scaled_Destination',
       'ID_Node_Scaled_Destination', 'ID_NUTS3_Scaled_Destination'],
      dtype='object')

In [ ]:


# Calcul de la somme des valeurs de la colonne "Max_Yearly_Trips"
somme_max_yearly_trips = Flux_Data['Traffic_flow_trucks_2019'].sum()*2

# Affichage de la somme
print("Somme des valeurs de Max_Yearly_Trips :", somme_max_yearly_trips)

Somme des valeurs de Max_Yearly_Trips : 49235980.0


## Création corridor_directionnels

In [ ]:
#On créé les corridors directionnels
Flux_Data['corridor_directionnel'] = Flux_Data.apply(lambda row: f"{row['Name_Scaled_Origin']} > {row['Name_Scaled_Destination']}", axis=1)


In [ ]:
nombre_de_lignes = Flux_Data.shape[0]
print(f"Le nombre de lignes dans Flux_Data est : {nombre_de_lignes}")

Le nombre de lignes dans Flux_Data est : 65970


## Selection du meilleur Path

In [ ]:
def custom_edge_path(series):

    matching_row = series[(series['ID_Node_Original_Origin'] == series['ID_Node_Scaled_Origin']) &
                          (series['ID_Node_Original_Destination'] == series['ID_Node_Scaled_Destination'])]


    if not matching_row.empty:
        return matching_row.iloc[0]['Edge_path_E_road']
    else:
        min_distance = np.inf
        selected_edge_path = None


        for _, row in series.iterrows():
            distance_term_origin = Nodes_Data.loc[(Nodes_Data['ID_Node_Original'] == row['ID_Node_Original_Origin']) & (Nodes_Data['ID_Node_Scaled'] == row['ID_Node_Scaled_Origin']), 'Distance_Term'].values[0]
            distance_term_destination = Nodes_Data.loc[(Nodes_Data['ID_Node_Original'] == row['ID_Node_Original_Destination']) & (Nodes_Data['ID_Node_Scaled'] == row['ID_Node_Scaled_Destination']), 'Distance_Term'].values[0]
            distance_term = distance_term_origin + distance_term_destination
            if distance_term < min_distance:
                min_distance = distance_term
                selected_edge_path = row['Edge_path_E_road']


        return selected_edge_path

In [ ]:
Path = Flux_Data.groupby('corridor_directionnel').apply(custom_edge_path)

In [ ]:
Path2 = pd.DataFrame(Path.reset_index())
Path2.columns = ['corridor_directionnel', 'Edge_Path']

In [ ]:
nombre_de_lignes = Path2.shape[0]
print(f"Le nombre de lignes dans Path2 est : {nombre_de_lignes}")

Le nombre de lignes dans Path2 est : 13385


In [ ]:
import pandas as pd

# Maintenant, filtrez les lignes où 'Edge_Path' est None
result = Path2.loc[Path2['Edge_Path']==None]

# Affichez le résultat
print(result)


Empty DataFrame
Columns: [corridor_directionnel, Edge_Path]
Index: []


In [ ]:
Path2.head()

,corridor_directionnel,Edge_Path
0,Hub_Aarhus(DK) > Hub_Algésiras(ES),"[1001995, 1306619, 1001994, 1001992, 1002144, ..."
1,Hub_Aarhus(DK) > Hub_Alicante - Crevillent(ES),"[1024582, 1024587, 1036120, 2610407, 1030885, ..."
2,Hub_Aarhus(DK) > Hub_Amiens(FR),"[1012443, 1019982, 1016502, 1019988, 1012462, ..."
3,Hub_Aarhus(DK) > Hub_Amsterdam(NL),"[1024582, 1024587, 1036120, 2610407, 1030885, ..."
4,Hub_Aarhus(DK) > Hub_Anvers(BE),"[1021837, 1015554, 1306314, 1306315, 1306275, ..."


In [ ]:
Flux_Data_Directionnel = Flux_Data.groupby('corridor_directionnel').agg({
    'Traffic_flow_trucks_2019': 'sum',
    'Traffic_flow_tons_2019': 'sum',
    'Total_Distance_PTV': 'mean',
    'ID_Node_Scaled_Origin': 'mean',
    'ID_Node_Scaled_Destination': 'mean',
}).reset_index()


In [ ]:
nombre_de_lignes = Flux_Data_Directionnel.shape[0]
print(f"Le nombre de lignes dans Path2 est : {nombre_de_lignes}")

Le nombre de lignes dans Path2 est : 13385


In [ ]:
# Fusionner les deux tables en utilisant la colonne 'corridor_directionnel' comme clé de fusion
resultat_fusion = Flux_Data_Directionnel.merge(Path2[['corridor_directionnel', 'Edge_Path']], on='corridor_directionnel', how='left')

# Afficher la table résultante
resultat_fusion.head()


,corridor_directionnel,Traffic_flow_trucks_2019,Traffic_flow_tons_2019,Total_Distance_PTV,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path
0,Hub_Aarhus(DK) > Hub_Algésiras(ES),83.75,1139,3089.000000,116479.0,101665.0,"[1001995, 1306619, 1001994, 1001992, 1002144, ..."
1,Hub_Aarhus(DK) > Hub_Alicante - Crevillent(ES),462.50,6290,2543.802667,116479.0,101570.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ..."
2,Hub_Aarhus(DK) > Hub_Amiens(FR),415.00,5644,1104.327500,116479.0,109273.0,"[1012443, 1019982, 1016502, 1019988, 1012462, ..."
3,Hub_Aarhus(DK) > Hub_Amsterdam(NL),1702.50,23154,791.394250,116479.0,196086.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ..."
4,Hub_Aarhus(DK) > Hub_Anvers(BE),6423.75,87363,858.130357,116479.0,111042.0,"[1021837, 1015554, 1306314, 1306315, 1306275, ..."


In [ ]:
nombre_de_lignes = resultat_fusion.shape[0]
print("Nombre de lignes dans la table resulat_fusion :", nombre_de_lignes)

Nombre de lignes dans la table resulat_fusion : 13385


In [ ]:
Flux_Data_Directionnel = resultat_fusion

In [ ]:


# Calcul de la somme des valeurs de la colonne "Max_Yearly_Trips"
somme_max_yearly_trips = Flux_Data_Directionnel['Traffic_flow_trucks_2019'].sum()*2

# Affichage de la somme
print("Somme des valeurs de Max_Yearly_Trips :", somme_max_yearly_trips)

Somme des valeurs de Max_Yearly_Trips : 49235980.0


## Création Corridors génériques

In [ ]:
# Fonction pour créer le corridor générique en ordre alphabétique
def create_generic_corridor(row):
    # Séparez le corridor en deux parties en utilisant le caractère ">"
    parts = row['corridor_directionnel'].split(' > ')

    # Triez les parties pour obtenir un ordre standardisé (A > B)
    parts.sort()

    # Rejoignez les parties pour créer le corridor générique
    return ' <> '.join(parts)

# Appliquez la fonction pour créer la colonne "Corridor Générique" dans Flux_Data_Directionnel
Flux_Data_Directionnel['Corridor_Générique'] = Flux_Data_Directionnel.apply(create_generic_corridor, axis=1)

# Triez le dataframe Flux_Data_Directionnel en utilisant la colonne "Corridor Générique"
Flux_Data_Directionnel.sort_values(by='Corridor_Générique', inplace=True)

# Réinitialisez l'index si nécessaire
Flux_Data_Directionnel.reset_index(drop=True, inplace=True)


In [ ]:
Flux_Data_Directionnel.columns

Index(['corridor_directionnel', 'Traffic_flow_trucks_2019',
       'Traffic_flow_tons_2019', 'Total_Distance_PTV', 'ID_Node_Scaled_Origin',
       'ID_Node_Scaled_Destination', 'Edge_Path', 'Corridor_Générique'],
      dtype='object')

In [ ]:
import pandas as pd

# Supposez que vous ayez déjà chargé vos données dans un DataFrame appelé df

# Comptez le nombre de lignes avec des erreurs, des valeurs vides ou "none" dans la colonne "Edge path"
count_errors = Flux_Data_Directionnel[Flux_Data_Directionnel['Edge_Path'].isna() | (Flux_Data_Directionnel['Edge_Path'] == 'none') | (Flux_Data_Directionnel['Edge_Path'].str.lower() == 'erreur')].shape[0]

# Affichez le résultat
print(f"Nombre de lignes avec erreur, valeur vide ou 'none' dans la colonne 'Edge path': {count_errors}")


Nombre de lignes avec erreur, valeur vide ou 'none' dans la colonne 'Edge path': 0


In [ ]:
import numpy as np

# Créez une fonction personnalisée pour obtenir la première valeur non vide de 'Edge_path_E_road'
def first_non_empty(series):
    non_empty_values = series.dropna()
    if len(non_empty_values) > 0:
        return non_empty_values.iloc[0]
    else:
        return np.nan

# Créez un dictionnaire de fonctions d'agrégation pour chaque colonne
aggregation_functions = {
    'Traffic_flow_tons_2019': 'min',
    'Traffic_flow_trucks_2019': 'min',
    'Total_Distance_PTV': 'mean',
    'ID_Node_Scaled_Origin': 'min',
    'ID_Node_Scaled_Destination': 'max',
    'Edge_Path': first_non_empty  # Appliquez la fonction personnalisée à 'Edge_path_E_road'
}

# Appliquez la fonction groupby en utilisant le dictionnaire d'agrégation
Flux_Data_Directionnel_grouped = Flux_Data_Directionnel.groupby('Corridor_Générique').agg(aggregation_functions)

# Réinitialisez l'index si nécessaire
Flux_Data_Directionnel_grouped.reset_index(inplace=True)

# Créez une copie du DataFrame résultant pour la nouvelle table Flux_Scaled
Flux_Scaled = Flux_Data_Directionnel_grouped

# Affichez le DataFrame Flux_Scaled avec les colonnes agrégées et 'Edge_path_E_road'
print(Flux_Scaled.columns)


Index(['Corridor_Générique', 'Traffic_flow_tons_2019',
       'Traffic_flow_trucks_2019', 'Total_Distance_PTV',
       'ID_Node_Scaled_Origin', 'ID_Node_Scaled_Destination', 'Edge_Path'],
      dtype='object')


In [ ]:
nombre_de_lignes = Flux_Data_Directionnel.shape[0]
print("Nombre de lignes dans la table Flux_Data_Directionnel :", nombre_de_lignes)

Nombre de lignes dans la table Flux_Data_Directionnel : 13385


In [ ]:


# Calcul de la somme des valeurs de la colonne "Max_Yearly_Trips"
somme_max_yearly_trips = Flux_Scaled['Traffic_flow_trucks_2019'].sum()*2

# Affichage de la somme
print("Somme des valeurs de Max_Yearly_Trips :", somme_max_yearly_trips)

Somme des valeurs de Max_Yearly_Trips : 19743362.5


In [ ]:
#On vérifie qu'il y a bien la moitié du nombre de ligne que Flux_Directionnel
nombre_de_lignes = len(Flux_Scaled)
print(nombre_de_lignes)


6789


In [ ]:
# Renommez les colonnes
Flux_Scaled.rename(columns={'Traffic_flow_tons_2019': 'Max_Yearly_Tons', 'Traffic_flow_trucks_2019': 'Max_Yearly_Trips'}, inplace=True)
Flux_Scaled.rename(columns={'Total_Distance_PTV': 'Total_Distance'}, inplace=True)
# Multipliez les valeurs par deux
Flux_Scaled['Max_Yearly_Tons'] *= 2
Flux_Scaled['Max_Yearly_Trips'] *= 2


In [ ]:
#On convertit les colonnes de la table en entiers
Edges_Data['Network_Edge_ID'] = Edges_Data['Network_Edge_ID'].astype(int)
Edges_Data['Network_Node_A_ID'] = Edges_Data['Network_Node_A_ID'].astype(int)
Edges_Data['Network_Node_B_ID'] = Edges_Data['Network_Node_B_ID'].astype(int)


In [ ]:
Flux_Scaled.columns

Index(['Corridor_Générique', 'Max_Yearly_Tons', 'Max_Yearly_Trips',
       'Total_Distance', 'ID_Node_Scaled_Origin', 'ID_Node_Scaled_Destination',
       'Edge_Path'],
      dtype='object')

In [ ]:
# Supposons que votre DataFrame s'appelle df
Flux_Scaled.rename(columns={'Total_Distance': 'Distance_Direct_ETIS_plus'}, inplace=True)


In [ ]:


# Calcul de la somme des valeurs de la colonne "Max_Yearly_Trips"
somme_max_yearly_trips = Flux_Scaled['Max_Yearly_Trips'].sum()

# Affichage de la somme
print("Somme des valeurs de Max_Yearly_Trips :", somme_max_yearly_trips)

Somme des valeurs de Max_Yearly_Trips : 19743362.5


In [ ]:

# Vérifier les doublons dans la colonne "Corridor_Générique"
doublons = Flux_Scaled['Corridor_Générique'].duplicated()

# Afficher les lignes avec des doublons
doublons_df = Flux_Scaled[doublons]
print("Lignes avec des doublons :")
print(doublons_df)


Lignes avec des doublons :
Empty DataFrame
Columns: [Corridor_Générique, Max_Yearly_Tons, Max_Yearly_Trips, Distance_Direct_ETIS_plus, ID_Node_Scaled_Origin, ID_Node_Scaled_Destination, Edge_Path]
Index: []


In [ ]:
Flux_Data.columns

Index(['ID_origin_NUTS3', 'Name_origin_NUTS3', 'ID_destination_NUTS3',
       'Name_destination_NUTS3', 'Edge_path_E_road',
       'Distance_from_origin_region_to_E_road', 'Distance_within_E_road',
       'Distance_from_E_road_to_destination_region', 'Total_distance_Mendeley',
       'Traffic_flow_trucks_2019', 'Traffic_flow_tons_2019',
       'Corridor_générique', 'Total_Distance_PTV', 'Eligibilité ',
       'ID_Node_Original_Origin', 'Name_Scaled_Origin',
       'ID_Node_Scaled_Origin', 'ID_NUTS3_Scaled_Origin',
       'ID_Node_Original_Destination', 'Name_Scaled_Destination',
       'ID_Node_Scaled_Destination', 'ID_NUTS3_Scaled_Destination',
       'corridor_directionnel'],
      dtype='object')

# Table Terminaux

In [ ]:
import pandas as pd
import ast

# Supposons que vous ayez déjà un DataFrame df avec une colonne "Edge_Pth" contenant des listes sous forme de chaînes de caractères

Flux_Scaled['Edge_Path'] =Flux_Scaled['Edge_Path'].apply(ast.literal_eval)

In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path
0,Hub_Aarhus(DK) <> Hub_Algésiras(ES),2278,167.5,3089.000000,101665.0,116479.0,"[1001995, 1306619, 1001994, 1001992, 1002144, ..."
1,Hub_Aarhus(DK) <> Hub_Alicante - Crevillent(ES),12580,925.0,2543.802667,101570.0,116479.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ..."
2,Hub_Aarhus(DK) <> Hub_Amiens(FR),11288,830.0,1104.327500,109273.0,116479.0,"[1012443, 1019982, 1016502, 1019988, 1012462, ..."
3,Hub_Aarhus(DK) <> Hub_Amsterdam(NL),46308,3405.0,791.394250,116479.0,196086.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ..."
4,Hub_Aarhus(DK) <> Hub_Anvers(BE),174726,12847.5,858.130357,111042.0,116479.0,"[1021837, 1015554, 1306314, 1306315, 1306275, ..."


In [ ]:
import pandas as pd

# Supposons que vous ayez déjà un DataFrame df avec une colonne "Edge Path" contenant des listes d'arêtes

# Définir votre fonction get_node_path
def get_node_path(edge_path):
    node_path = []
    for edge_id in edge_path:
        edge_info = Edges_Data[Edges_Data['Network_Edge_ID'] == edge_id]
        if not edge_info.empty:
            node_path.extend([edge_info.iloc[0]['Network_Node_A_ID'], edge_info.iloc[0]['Network_Node_B_ID']])
    return node_path

# Appliquer la fonction à la colonne "Edge Path" et créer la colonne "Nodes Path"
Flux_Scaled['Nodes_Path'] = Flux_Scaled['Edge_Path'].apply(get_node_path)


In [ ]:
nombre_de_lignes = Terminaux_Data.shape[0]
print("Le nombre de lignes dans Terminaux_Data est :", nombre_de_lignes)

Le nombre de lignes dans Terminaux_Data est : 180


In [ ]:
ligne_caceres = Terminaux_Data.loc[Terminaux_Data['Terminal_Name'] == 'Caceres']
ligne_caceres

,Terminal_Name,Terminal_Lat,Terminal_Long,Step,Go Live Planned,Node_Lat,Node_Long,Transit,Network_Node_ID,Country,List_Nodes_Hub
31,Caceres,39.473056,-6.371111,S1_Ph3,2030,39.469517,-6.383135,['Merida'],101418,ES,"[101418, 101421, 101396, 101397, 190384, 19038..."


In [ ]:
Flux_Scaled['Nodes_Path'] = Flux_Scaled['Nodes_Path'].apply(lambda path_list: [int(x) for x in path_list])

In [ ]:
#On fait une vérification sur la longueur des deux listes
length_of_list_node = len(Flux_Scaled.at[3, 'Nodes_Path'])
print(length_of_list_node)
length_of_list_edges = len(Flux_Scaled.at[3, 'Edge_Path'])
print(length_of_list_edges)

268
134


In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path
0,Hub_Aarhus(DK) <> Hub_Algésiras(ES),2278,167.5,3089.000000,101665.0,116479.0,"[1001995, 1306619, 1001994, 1001992, 1002144, ...","[101660, 101661, 101660, 190311, 101658, 19031..."
1,Hub_Aarhus(DK) <> Hub_Alicante - Crevillent(ES),12580,925.0,2543.802667,101570.0,116479.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ...","[130110, 116498, 130111, 130110, 130111, 26235..."
2,Hub_Aarhus(DK) <> Hub_Amiens(FR),11288,830.0,1104.327500,109273.0,116479.0,"[1012443, 1019982, 1016502, 1019988, 1012462, ...","[109246, 109247, 109247, 122670, 122670, 10920..."
3,Hub_Aarhus(DK) <> Hub_Amsterdam(NL),46308,3405.0,791.394250,116479.0,196086.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ...","[130110, 116498, 130111, 130110, 130111, 26235..."
4,Hub_Aarhus(DK) <> Hub_Anvers(BE),174726,12847.5,858.130357,111042.0,116479.0,"[1021837, 1015554, 1306314, 1306315, 1306275, ...","[111033, 111055, 111033, 195923, 195923, 19592..."


In [ ]:
# Créez un dictionnaire pour mapper les IDs de nœuds aux noms de terminaux
# Créez un dictionnaire pour mapper les node_ids aux noms 'Name_Scaled'
node_id_to_name_scaled1 = {}

# Parcourir les lignes de Nodes_Data
for index, row in Nodes_Data.iterrows():
    node_id_original = row['ID_Node_Original']
    name_scaled = row['Name_City']

    # Si le node_id_original n'est pas nul, l'ajouter au dictionnaire
    if not pd.isnull(node_id_original):
        node_id_to_name_scaled1[node_id_original] = name_scaled

# Fonction pour créer le routing de terminaux sans redondance
def create_terminal_routing(node_ids):
    terminal_routing = []
    seen_terminals = set()  # Utilisez un ensemble pour éviter les redondances
    for node_id in node_ids:
        terminal_name = node_id_to_name_scaled1.get(node_id, '')
        if terminal_name and terminal_name not in seen_terminals:
            terminal_routing.append(terminal_name)
            seen_terminals.add(terminal_name)
    return '<>'.join(terminal_routing)  # Concatène les noms de terminaux sans redondance avec '<>'

# Appliquez la fonction aux données dans la colonne 'Nodes_Path'
Flux_Scaled['Routing'] = Flux_Scaled['Nodes_Path'].apply(create_terminal_routing)





In [ ]:
distance_matrix= pd.read_excel("drive/MyDrive/ECTN_Routing/distance_matrix_3.xlsx")

In [ ]:
distance_matrix

,Varsovie,Almería,Dresde,Orense,Oradea,Brest,Brême,Vienne,Marseille - Aix-en-Provence,Peristéri,...,Albacete - San Clemente,Namur,Miranda De Ebro,Caceres,Craiova,Lucerne,Merida,Gospic,Struzani,Zilina
0,0.000,3149.186,647.505,3087.893,1014.772,2137.287,952.609,705.184,2023.847,2352.230,...,2917.790,1297.969,2545.403,3074.764,1530.184,1402.669,3138.823,1243.673,1195.703,495.419
1,3149.186,0.000,2528.164,1081.523,3036.813,1887.294,2488.096,2557.854,1269.198,3854.742,...,420.226,2102.111,890.172,679.074,3190.864,1804.216,608.323,2436.881,2596.052,2837.977
2,647.505,2528.164,0.000,2492.517,1011.794,1618.309,491.297,481.178,1407.861,2132.948,...,2301.804,778.991,1950.027,2479.388,1310.902,786.683,2543.447,1019.667,976.421,558.662
3,3087.893,1081.523,2492.517,0.000,3150.692,1537.946,2272.944,2612.791,1383.077,3968.621,...,705.686,1797.844,529.313,531.615,3304.743,1859.153,595.674,2550.760,2709.931,2857.285
4,1014.772,3036.813,1011.794,3150.692,0.000,2404.137,1495.624,579.022,1765.397,1420.057,...,2800.954,1666.133,2618.647,3132.958,531.654,1383.495,3171.652,869.208,486.213,524.603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,1402.669,1804.216,786.683,1859.153,1383.495,1150.488,862.635,800.219,679.028,2350.178,...,1564.312,600.644,1331.156,1861.263,1699.940,0.000,1930.340,922.654,1081.825,1082.339
176,3138.823,608.323,2543.447,595.674,3171.652,1604.411,2324.452,2688.641,1408.281,3995.717,...,486.075,1856.650,594.592,71.861,3301.103,1930.340,0.000,2575.467,2734.638,2923.837
177,1243.673,2436.881,1019.667,2550.760,869.208,2145.860,1428.030,547.736,1168.806,1676.462,...,2196.977,1408.996,2022.763,2536.613,981.848,922.654,2575.467,0.000,413.080,811.453
178,1195.703,2596.052,976.421,2709.931,486.213,2193.413,1475.583,542.377,1327.977,1265.924,...,2356.148,1456.549,2181.934,2695.784,571.310,1081.825,2734.638,413.080,0.000,724.853


In [ ]:
def get_column_number(city_name, cities_list):
    try:
        return cities_list.get_loc(city_name)
    except KeyError:
        return None

In [ ]:
import re

def extract_city_and_country(corridor):
    matches = re.findall(r'Hub_([\w\s-]+)\((\w+)\)', corridor)
    cities = []
    for match in matches:
        cities.append(match[0])
    return cities


In [ ]:
import pandas as pd

# Supposons que vous ayez déjà chargé votre DataFrame 'df' et la matrice 'matrice_distances'

# Fonction pour calculer la distance entre deux villes
def find_distance(row):
    route = row['Corridor_Générique']
    cities = extract_city_and_country(route)
    if len(cities) >= 2:
        start_city = cities[0]
        end_city = cities[-1]
        start_col = get_column_number(start_city, distance_matrix.columns)
        end_col = get_column_number(end_city, distance_matrix.columns)
        distance = distance_matrix.loc[start_col, end_city]
        return distance
    else:
        return None

# Appliquez la fonction à chaque ligne du DataFrame pour créer la colonne "Distance_Direct"
Flux_Scaled['Distance_Direct'] = Flux_Scaled.apply(find_distance, axis=1)


In [ ]:
# Supposons que votre DataFrame s'appelle df
lignes_avec_distance_nulle = Flux_Scaled[Flux_Scaled['Distance_Direct'] == None]

# Affichez les lignes avec distance nulle
lignes_avec_distance_nulle

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct


In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct
0,Hub_Aarhus(DK) <> Hub_Algésiras(ES),2278,167.5,3089.000000,101665.0,116479.0,"[1001995, 1306619, 1001994, 1001992, 1002144, ...","[101660, 101661, 101660, 190311, 101658, 19031...",Algésiras<>Grenade<>Almería<>Alicante - Crevil...,3127.138
1,Hub_Aarhus(DK) <> Hub_Alicante - Crevillent(ES),12580,925.0,2543.802667,101570.0,116479.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ...","[130110, 116498, 130111, 130110, 130111, 26235...",Aarhus<>Hambourg<>Brême<>Bielefeld<>Enschede<>...,2662.102
2,Hub_Aarhus(DK) <> Hub_Amiens(FR),11288,830.0,1104.327500,109273.0,116479.0,"[1012443, 1019982, 1016502, 1019988, 1012462, ...","[109246, 109247, 109247, 122670, 122670, 10920...",Amiens<>Lille<>Gand<>Namur<>Maastricht<>Cologn...,1132.291
3,Hub_Aarhus(DK) <> Hub_Amsterdam(NL),46308,3405.0,791.394250,116479.0,196086.0,"[1024582, 1024587, 1036120, 2610407, 1030885, ...","[130110, 116498, 130111, 130110, 130111, 26235...",Aarhus<>Hambourg<>Brême<>Emmen<>Enschede<>Nimè...,795.323
4,Hub_Aarhus(DK) <> Hub_Anvers(BE),174726,12847.5,858.130357,111042.0,116479.0,"[1021837, 1015554, 1306314, 1306315, 1306275, ...","[111033, 111055, 111033, 195923, 195923, 19592...",Anvers<>Eindhoven<>Rotterdam<>Amsterdam<>Nimèg...,878.510


In [ ]:
Flux_Scaled= Flux_Scaled.sort_values(by='Max_Yearly_Trips', ascending=False)

In [ ]:
#On change à la main le routing des corridors biarres
import pandas as pd

# Liste des noms de corridors génériques à mettre à jour
corridors_a_mettre_a_jour = [
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rotterdam(NL)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rotterdam(NL)',
    'Hub_Anvers(BE) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Anvers(BE) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Orléans(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Orléans(FR)'
]

# Nouvelles valeurs pour la colonne "Routing"
nouvelles_valeurs_routing = [
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans'
]

# Parcourez le DataFrame et mettez à jour les valeurs de la colonne "Routing" en fonction des noms de corridors génériques
for corridor, nouvelle_valeur in zip(corridors_a_mettre_a_jour, nouvelles_valeurs_routing):
    Flux_Scaled.loc[Flux_Scaled['Corridor_Générique'] == corridor, 'Routing'] = nouvelle_valeur


In [ ]:
# Créez un dictionnaire pour mapper les IDs de nœuds aux noms de terminaux
# Créez un dictionnaire pour mapper les node_ids aux noms 'Name_Scaled'
node_id_to_name_scaled = {}

for index, row in Nodes_Data.iterrows():
    node_id_original = row['ID_Node_Original']
    name_scaled = row['Name_City']
    distance_term = row['Distance_Term']
    transit_authorization = Terminaux_Data.loc[Terminaux_Data['Terminal_Name'] == name_scaled, 'Transit'].values[0]!="N"

    # Si le node_id_original n'est pas nul, l'ajouter au dictionnaire
    if not pd.isnull(node_id_original) and distance_term<=60 and transit_authorization:
        node_id_to_name_scaled[node_id_original] = name_scaled


In [ ]:
def create_terminal_routing_bis1(node_ids):
    terminal_routing = []
    seen_terminals = set()  # Utilisez un ensemble pour éviter les redondances

    # Parcourir les node_ids intermédiaires
    for node_id in node_ids:
        terminal_name = node_id_to_name_scaled.get(node_id, '')
        if terminal_name and terminal_name not in seen_terminals:
            terminal_routing.append(terminal_name)
            seen_terminals.add(terminal_name)


    return '<>'.join(terminal_routing)  # Concatène les noms de terminaux sans redondance avec '<>'

# Appliquez la fonction aux données dans la colonne 'Nodes_Path'
Flux_Scaled['Routing_Bis1'] = Flux_Scaled['Nodes_Path'].apply(create_terminal_routing_bis1)


In [ ]:
#On change à la main le routing des corridors biarres
import pandas as pd

# Liste des noms de corridors génériques à mettre à jour
corridors_a_mettre_a_jour = [
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rotterdam(NL)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rotterdam(NL)',
    'Hub_Anvers(BE) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Anvers(BE) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Orléans(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Orléans(FR)'
]

# Nouvelles valeurs pour la colonne "Routing"
nouvelles_valeurs_routing = [
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans',
]


# Parcourez le DataFrame et mettez à jour les valeurs de la colonne "Routing" en fonction des noms de corridors génériques
for corridor, nouvelle_valeur in zip(corridors_a_mettre_a_jour, nouvelles_valeurs_routing):
    Flux_Scaled.loc[Flux_Scaled['Corridor_Générique'] == corridor, 'Routing_Bis1'] = nouvelle_valeur

In [ ]:
lignes_sans_routing_bis1 = Flux_Scaled.loc[Flux_Scaled['Routing_Bis1'].isnull() | (Flux_Scaled['Routing_Bis1'] == '')]
lignes_sans_routing_bis1


,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1
78,Hub_Aarhus(DK) <> Hub_Oslo(NO),329664,24240.0,858.6498,116479.0,250487.0,"[2500924, 2610062, 2610093, 2610094, 1036108, ...","[130103, 124372, 130103, 262750, 262751, 26275...",Aarhus<>Oslo,900.764,


In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,Barcelona - Sant andreu de la barca<>Montpelli...
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...


In [ ]:
import pandas as pd

# Supposons que vous ayez déjà chargé votre DataFrame Scaled_Data

def combine_routing(row):
    routing_list = row['Routing']
    routing_bis1 = row['Routing_Bis1']

    # Divisez les chaînes de routage en listes en utilisant '<>' comme séparateur
    routing_list = routing_list.split('<>')
    routing_bis1_list = routing_bis1.split('<>')

    # Vérifiez le premier élément
    if routing_list[0] != routing_bis1_list[0]:
        routing_bis1 = f"{routing_list[0]}<>{routing_bis1}"

    # Vérifiez le dernier élément
    if routing_list[-1] != routing_bis1_list[-1]:
        routing_bis1 = f"{routing_bis1}<>{routing_list[-1]}"

    return routing_bis1

# Appliquez la fonction à chaque ligne de votre DataFrame pour créer la nouvelle colonne
Flux_Scaled['Routing_Bis2'] = Flux_Scaled.apply(combine_routing, axis=1)


In [ ]:
#On change à la main le routing des corridors biarres
import pandas as pd

# Liste des noms de corridors génériques à mettre à jour
corridors_a_mettre_a_jour = [
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rotterdam(NL)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rotterdam(NL)',
    'Hub_Anvers(BE) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Anvers(BE) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Orléans(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Orléans(FR)'
]

# Nouvelles valeurs pour la colonne "Routing"
nouvelles_valeurs_routing = [
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans',
]


# Parcourez le DataFrame et mettez à jour les valeurs de la colonne "Routing" en fonction des noms de corridors génériques
for corridor, nouvelle_valeur in zip(corridors_a_mettre_a_jour, nouvelles_valeurs_routing):
    Flux_Scaled.loc[Flux_Scaled['Corridor_Générique'] == corridor, 'Routing_Bis2'] = nouvelle_valeur

In [ ]:
import re

def extract_city_and_country(corridor):
    matches = re.findall(r'Hub_([\w\s-]+)\((\w+)\)', corridor)
    cities = []
    for match in matches:
        cities.append(match[0])
    return cities

In [ ]:
def update_routing(row):
    routing = row['Routing_Bis2'].split('<>')
    corridor = row['Corridor_Générique']
    cities = extract_city_and_country(corridor)

    # Tant que le premier élément de "Routing" n'est pas l'une des villes extraites, le supprimer
    while routing[0] not in cities:
        routing.pop(0)

    # Tant que le dernier élément de "Routing" n'est pas l'une des villes extraites, le supprimer
    while routing[-1] not in cities:
        routing.pop(-1)

    # Rejoindre la liste mise à jour pour former la nouvelle valeur de "Routing"
    return '<>'.join(routing)

# Appliquer la fonction d'actualisation à la colonne "Routing"
Flux_Scaled['Routing_Bis2'] = Flux_Scaled.apply(update_routing, axis=1)


In [ ]:
def commence_par_crochet(texte):
    # Vérifie si le texte commence par "["
    return texte.startswith("[")


In [ ]:
def transformer_routing(row):
    routing_initial = row['Routing_Bis2'].split('<>')
    nouveau_routing_bis_3 = [routing_initial[0]]

    for i in range(len(routing_initial) - 1):
      terminal_actuel = nouveau_routing_bis_3[i]

    # Recherchez les correspondances du Terminal_Name
      matches = Terminaux_Data.loc[Terminaux_Data['Terminal_Name'] == terminal_actuel, 'Transit']

    # Vérifiez s'il y a des correspondances
      if not matches.empty:
        decision_terminal_actuel = matches.values[0]
        if commence_par_crochet(decision_terminal_actuel):
            if routing_initial[i + 1] not in decision_terminal_actuel:
                nouveau_routing_bis_3.append(routing_initial[i + 1])
            else:
                nouveau_routing_bis_3.append(terminal_actuel)
        else:
            nouveau_routing_bis_3.append(routing_initial[i + 1])

    terminaux_uniques = []

    for terminal in nouveau_routing_bis_3:
        if terminal not in terminaux_uniques:
            terminaux_uniques.append(terminal)

    decision_terminal_final = Terminaux_Data.loc[Terminaux_Data['Terminal_Name'] == routing_initial[-1], 'Transit'].values[0]

    if decision_terminal_final == "Y" or decision_terminal_final == "N":
        terminaux_uniques.append(routing_initial[-1])
    else:
        i = len(terminaux_uniques) - 1
        while terminaux_uniques[i] in decision_terminal_final:
            terminaux_uniques.pop(-1)
            i = i - 1
        terminaux_uniques.append(routing_initial[-1])

    return '<>'.join(terminaux_uniques)



In [ ]:
# Appliquer la fonction à chaque ligne du DataFrame et créer la colonne Routing_Bis3
Flux_Scaled['Routing_Bis3'] = Flux_Scaled.apply(transformer_routing, axis=1)

# Afficher le DataFrame avec la nouvelle colonne Routing_Bis3
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,Barcelona - Sant andreu de la barca<>Montpelli...,Barcelona - Sant andreu de la barca<>Montpelli...,Barcelona - Sant andreu de la barca<>Montpelli...
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,Milan - assago<>Lucerne<>Mulhouse<>Nancy<>Luxe...,Milan - assago<>Lucerne<>Mulhouse<>Nancy<>Luxe...
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,Moers<>Francfort - Mayence<>Nuremberg - Fürth<...


In [ ]:
lignes_sans_routing_bis1 = Flux_Scaled.loc[Flux_Scaled['Routing_Bis3'].isnull() | (Flux_Scaled['Routing_Bis3'] == '')]
lignes_sans_routing_bis1

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3


In [ ]:
#On change à la main le routing des corridors biarres
import pandas as pd

# Liste des noms de corridors génériques à mettre à jour
corridors_a_mettre_a_jour = [
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Lille(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Moissy-Cramayel(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Paris - Vémars(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rouen(FR)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Rotterdam(NL)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Rotterdam(NL)',
    'Hub_Anvers(BE) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Anvers(BE) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona -Mollet del Vallès(ES)',
    'Hub_Amiens(FR) <> Hub_Barcelona - Sant andreu de la barca(ES)',
    'Hub_Barcelona -Mollet del Vallès(ES) <> Hub_Orléans(FR)',
    'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Orléans(FR)'
]

# Nouvelles valeurs pour la colonne "Routing"
nouvelles_valeurs_routing = [
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Lille',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Moissy-Cramayel',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Paris - Vémars',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Rouen',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers<>Rotterdam',
    'Barcelona -Mollet del Vallès<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Lyon - Saint-Priest<>Dijon<>Sommesous<>Anvers',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans<>Amiens',
    'Barcelona -Mollet del Vallès<>Montpellier<>Clermont-Ferrand<>Orléans',
    'Barcelona - Sant andreu de la barca<>Montpellier<>Clermont-Ferrand<>Orléans',
]


# Parcourez le DataFrame et mettez à jour les valeurs de la colonne "Routing" en fonction des noms de corridors génériques
for corridor, nouvelle_valeur in zip(corridors_a_mettre_a_jour, nouvelles_valeurs_routing):
    Flux_Scaled.loc[Flux_Scaled['Corridor_Générique'] == corridor, 'Routing_Bis3'] = nouvelle_valeur

In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,Barcelona - Sant andreu de la barca<>Montpelli...,Barcelona - Sant andreu de la barca<>Montpelli...,Barcelona - Sant andreu de la barca<>Montpelli...
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,Milan - assago<>Lucerne<>Mulhouse<>Nancy<>Luxe...,Milan - assago<>Lucerne<>Mulhouse<>Nancy<>Luxe...
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,Moers<>Francfort - Mayence<>Nuremberg - Fürth<...


In [ ]:
import requests
import pandas as pd
import json
from google.colab import drive


# URL de l'API PTV Developer
url = "https://api.myptv.com/routing/v1/routes"
# Clé d'API PTV Developer
api_key = "RVVfMDQzNjc0ODkwYTZkNGNiNzkyZGU4MDY2YmI5OTk0OTY6MWIyMjgyODctN2VhYS00NTRkLWI1ODAtYzU4ZjljZmNlMGIy"


def calculate_distance(waypoint1,waypoint2):

    waypoints_query1 = f"waypoints={waypoint1}"
    waypoints_query2 = f"waypoints={waypoint2}"
    url_with_params = f"{url}?{waypoints_query1}&{waypoints_query2}&options[trafficMode]=AVERAGE"

    headers = {
        'apiKey': api_key
    }

    response = requests.get(url_with_params, headers=headers)
    json_content = response.content
    response_data = json.loads(json_content)

    try:
        Distance = response_data['distance']
        Distance=Distance/1000
    except Exception as e:
        return "Erreur"
    return Distance

In [ ]:
def remove_duplicates(row):
    elements = row.split('<>')
    unique_elements = []

    for element in elements:
        if element not in unique_elements:
            unique_elements.append(element)

    return '<>'.join(unique_elements)

# Appliquez la fonction à la colonne 'Routing_Bis3'
Flux_Scaled['Routing_Bis3'] = Flux_Scaled['Routing_Bis3'].apply(remove_duplicates)



In [ ]:
def get_column_number(city_name, cities_list):
    try:
        return cities_list.get_loc(city_name)
    except KeyError:
        return None

In [ ]:
def get_reduced_distance(row):
    routing = row['Routing_Bis3'].split('<>')
    Distance = []
    for i in range(len(routing) - 1):
        start_city = routing[i]
        end_city = routing[i+1]

        start_col = get_column_number(start_city, distance_matrix.columns)
        end_col = get_column_number(end_city, distance_matrix.columns)

        if start_col is None or end_col is None:
            # Si l'une des villes n'a pas de correspondance dans la distance_matrix, calculez la distance autrement
            start_city_data = Terminaux_Data[Terminaux_Data['Terminal_Name'] == start_city]
            end_city_data = Terminaux_Data[Terminaux_Data['Terminal_Name'] == end_city]
            if not start_city_data.empty and not end_city_data.empty:
                start_city_lat = start_city_data['Node_Lat'].values[0]
                start_city_long = start_city_data['Node_Long'].values[0]
                end_city_lat = end_city_data['Node_Lat'].values[0]
                end_city_long = end_city_data['Node_Long'].values[0]
                waypoint1 = f"{start_city_lat},{start_city_long}"
                waypoint2 = f"{end_city_lat},{end_city_long}"
                distance = haversine((start_city_lat, start_city_long), (end_city_lat, end_city_long))
                distance = round(distance, 2)  # Arrondir à deux chiffres après la virgule

            else:
                return "Erreur"  # Aucune correspondance trouvée pour une ou les deux villes
        else:
            # Utilisez la distance à partir de la distance_matrix
            distance = distance_matrix.iloc[start_col, end_col]

        Distance.append(distance)

    return Distance




In [ ]:
 Flux_Scaled['Distance_InterHub'] = Flux_Scaled.apply(get_reduced_distance, axis=1)

In [ ]:
Test = Flux_Scaled.loc[Flux_Scaled['Distance_InterHub']=="Erreur"]
Test

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3,Distance_InterHub
78,Hub_Aarhus(DK) <> Hub_Oslo(NO),329664,24240.0,858.6498,116479.0,250487.0,"[2500924, 2610062, 2610093, 2610094, 1036108, ...","[130103, 124372, 130103, 262750, 262751, 26275...",Aarhus<>Oslo,900.764,,Aarhus<><>Oslo,Aarhus<><>Oslo,Erreur


In [ ]:
start_col = get_column_number("Oslo", distance_matrix.columns)
start_col

130

In [ ]:
def get_reduced_routing(row):
    routing = row['Routing_Bis3'].split('<>')
    Distance = row['Distance_InterHub']

    # Si la Distance est "Erreur", renvoyer "Erreur" directement
    if Distance == "Erreur":
        return "Erreur"
    Distance.append(3000)
    reduced_routing = [routing[0]]
    current_distance = 0

    for i in range(len(Distance) - 1):
        current_distance += Distance[i]
        if current_distance <= 350 and current_distance + Distance[i + 1] > 350:
            reduced_routing.append(routing[i + 1])
            current_distance = 0
        elif current_distance > 350:
            reduced_routing.append(routing[i + 1])
            current_distance = 0
    Distance.pop()


    # Si la boucle se termine sans erreurs, renvoyez le routing réduit
    return '<>'.join(reduced_routing)




In [ ]:
# Supposons que votre DataFrame s'appelle df
Flux_Scaled['Routing_Final'] = Flux_Scaled.apply(get_reduced_routing, axis=1)

In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3,Distance_InterHub,Routing_Final
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,Barcelona - Sant andreu de la barca<>Montpelli...,Barcelona - Sant andreu de la barca<>Montpelli...,Barcelona - Sant andreu de la barca<>Montpelli...,"[321.204, 304.084, 206.608, 257.183, 256.451]",Barcelona - Sant andreu de la barca<>Montpelli...
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,"[307.838, 114.66, 203.152, 240.296, 318.326]",Porto - Lagoa<>Benavente<>Miranda De Ebro<>Sar...
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,Milan - assago<>Lucerne<>Mulhouse<>Nancy<>Luxe...,Milan - assago<>Lucerne<>Mulhouse<>Nancy<>Luxe...,"[257.791, 131.127, 218.261, 131.547, 154.025, ...",Milan - assago<>Lucerne<>Nancy<>Namur<>Lille
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Orléan...,"[132.16, 189.351, 253.06, 219.774, 115.711, 19...",Pampelune<>Bordeaux<>Poitiers<>Paris - Moissy-...
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,Moers<>Francfort - Mayence<>Nuremberg - Fürth<...,"[258.001, 252.053, 346.615]",Moers<>Francfort - Mayence<>Nuremberg - Fürth<...


In [ ]:
Test = Flux_Scaled.loc[Flux_Scaled['Routing_Final']=="Erreur"]
Test

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3,Distance_InterHub,Routing_Final
78,Hub_Aarhus(DK) <> Hub_Oslo(NO),329664,24240.0,858.6498,116479.0,250487.0,"[2500924, 2610062, 2610093, 2610094, 1036108, ...","[130103, 124372, 130103, 262750, 262751, 26275...",Aarhus<>Oslo,900.764,,Aarhus<><>Oslo,Aarhus<><>Oslo,Erreur,Erreur


In [ ]:
def get_reduced_distance2(row):
    routing = row['Routing_Final'].split('<>')
    Distance = []
    for i in range(len(routing) - 1):
        start_city = routing[i]
        end_city = routing[i+1]

        start_col = get_column_number(start_city, distance_matrix.columns)
        end_col = get_column_number(end_city, distance_matrix.columns)

        if start_col is None or end_col is None:
            # Si l'une des villes n'a pas de correspondance dans la distance_matrix, calculez la distance autrement
            start_city_data = Terminaux_Data[Terminaux_Data['Terminal_Name'] == start_city]
            end_city_data = Terminaux_Data[Terminaux_Data['Terminal_Name'] == end_city]
            if start_city_data.empty or end_city_data.empty:
                return "Erreur"
            else:
              start_city_lat = start_city_data['Node_Lat'].values[0]
              start_city_long = start_city_data['Node_Long'].values[0]
              end_city_lat = end_city_data['Node_Lat'].values[0]
              end_city_long = end_city_data['Node_Long'].values[0]
              waypoint1 = f"{start_city_lat},{start_city_long}"
              waypoint2 = f"{end_city_lat},{end_city_long}"
              distance = haversine((start_city_lat,start_city_long), (end_city_lat,end_city_long))
              distance = round(distance, 2)  # Aucune correspondance trouvée pour une ou les deux villes
        else:
            # Utilisez la distance à partir de la distance_matrix
            distance = distance_matrix.iloc[start_col, end_col]

        Distance.append(distance)

    return Distance

# Appliquez la fonction aux données dans la colonne 'Routing_Bis'
Flux_Scaled['Distance_InterHub_Final'] = Flux_Scaled.apply(get_reduced_distance2, axis=1)

In [ ]:
Test = Flux_Scaled.loc[Flux_Scaled['Distance_InterHub_Final']=="Erreur"]
Test

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,Routing_Bis1,Routing_Bis2,Routing_Bis3,Distance_InterHub,Routing_Final,Distance_InterHub_Final


In [ ]:
import pandas as pd

# Supposons que vous ayez déjà chargé votre DataFrame avec la colonne 'Distance_Inter_Hub_Final'

# Créez une fonction pour compter le nombre de distances dans différentes catégories
def count_distances(distances, lower, upper):
    return sum(1 for distance in distances if lower <= distance <= upper)

# Créez une fonction pour calculer la moyenne des distances
def calculate_average_distance(distances):
    if len(distances) > 0:
        return sum(distances) / len(distances)
    else:
        return 0  # Ou une autre valeur par défaut que vous préférez

# Appliquez ces fonctions à chaque ligne de votre DataFrame pour créer les nouvelles colonnes
Flux_Scaled['<200'] = Flux_Scaled['Distance_InterHub_Final'].apply(lambda x: count_distances(x, 0, 200))
Flux_Scaled['[200,300]'] = Flux_Scaled['Distance_InterHub_Final'].apply(lambda x: count_distances(x, 200, 300))
Flux_Scaled['[300,350]'] = Flux_Scaled['Distance_InterHub_Final'].apply(lambda x: count_distances(x, 300, 350))
Flux_Scaled['>350'] = Flux_Scaled['Distance_InterHub_Final'].apply(lambda x: count_distances(x, 350, float('inf')))
Flux_Scaled['Distance_moyenne_segment'] = Flux_Scaled['Distance_InterHub_Final'].apply(calculate_average_distance)


In [ ]:
# Convertir la colonne 'Traffic_flow_trucks_2019' en float si elle est de type différent
Flux_Scaled['Max_Yearly_Trips'] = Flux_Scaled['Max_Yearly_Trips'].astype(float)


In [ ]:
# Supposons que vous avez une colonne spécifique que vous souhaitez convertir, par exemple 'Go Live Planned 1'

Terminaux_Data['Go Live Planned'] = pd.to_numeric(Terminaux_Data['Go Live Planned'], errors='coerce')

# L'argument 'errors' est réglé sur 'coerce' pour que les valeurs non numériques deviennent NaN


In [ ]:
Terminaux_Data['Step'] = Terminaux_Data['Step'].astype(str)


In [ ]:
import pandas as pd
import re  # Importez le module des expressions régulières

# Supposons que vous avez déjà chargé vos DataFrames Flux_Scaled et Terminaux_Data

# Fonction pour extraire le nombre de terminaux et la valeur maximale de 'Step' et 'Go Live Planned'
def extract_terminal_info(routing):
    if routing == 'Erreur':
        return 'Erreur', 'Erreur', 'Erreur'

    # Divisez la chaîne 'Routing' en terminaux
    terminals = routing.split('<>')

    # Initialisez les variables pour le nombre de terminaux, Step max et Go Live Planned max
    num_terminals = len(terminals)
    max_step = 0  # Initialisez avec 0 (la valeur la moins avancée)
    max_go_live_planned = 0  # Initialisez avec 0 (l'année la moins avancée)

    # Parcourez chaque terminal dans la liste
    for terminal in terminals:
        # Recherchez le terminal dans Terminaux_Data
        terminal_info = Terminaux_Data[Terminaux_Data['Terminal_Name'] == terminal]
        if not terminal_info.empty:
            step = terminal_info['Step'].values[0]
            go_live_planned = terminal_info['Go Live Planned'].values[0]

            # Utilisez une expression régulière pour extraire le numéro de phase et le numéro de phase
            step_match = re.match(r'S(\d+)_Ph(\d+)', step)
            if step_match:
                step_number = int(step_match.group(1) + step_match.group(2))
            else:
                step_number = 0

            # Mettez à jour 'max_step' si 'step_number' est plus avancé
            if step_number > max_step:
                max_step = step_number

            # Mettez à jour 'max_go_live_planned' si 'go_live_planned' est plus avancé
            if go_live_planned > max_go_live_planned:
                max_go_live_planned = go_live_planned

    return num_terminals, step, max_go_live_planned

# Appliquez la fonction pour créer les nouvelles colonnes
Flux_Scaled[['Nbe_Terminals', 'Step', 'Go_Live_Planned']] = Flux_Scaled['Routing_Final'].apply(extract_terminal_info).apply(pd.Series)



In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,Routing_Final,Distance_InterHub_Final,<200,"[200,300]","[300,350]",>350,Distance_moyenne_segment,Nbe_Terminals,Step,Go_Live_Planned
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,...,Barcelona - Sant andreu de la barca<>Montpelli...,"[321.204, 304.084, 206.608, 257.183, 256.451]",0,3,2,0,269.106000,6,S1_Ph2,2030.0
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,...,Porto - Lagoa<>Benavente<>Miranda De Ebro<>Sar...,"[307.838, 316.677, 240.296, 318.326]",0,1,3,0,295.784250,5,S1_Ph3,2030.0
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,...,Milan - assago<>Lucerne<>Nancy<>Namur<>Lille,"[257.791, 345.539, 281.48, 155.262]",1,2,1,0,260.018000,5,S1_Ph2,2030.0
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,...,Pampelune<>Bordeaux<>Poitiers<>Paris - Moissy-...,"[316.526, 253.06, 336.442, 264.553]",0,2,2,0,292.645250,5,S1_Ph2,2030.0
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,...,Moers<>Francfort - Mayence<>Nuremberg - Fürth<...,"[258.001, 252.053, 346.615]",0,2,1,0,285.556333,4,S1_Ph3,2030.0


# Type Produits

In [ ]:
Flux_Scaled.columns

Index(['Corridor_Générique', 'Max_Yearly_Tons', 'Max_Yearly_Trips',
       'Distance_Direct_ETIS_plus', 'ID_Node_Scaled_Origin',
       'ID_Node_Scaled_Destination', 'Edge_Path', 'Nodes_Path', 'Routing',
       'Distance_Direct', 'Routing_Bis1', 'Routing_Bis2', 'Routing_Bis3',
       'Distance_InterHub', 'Routing_Final', 'Distance_InterHub_Final', '<200',
       '[200,300]', '[300,350]', '>350', 'Distance_moyenne_segment',
       'Nbe_Terminals', 'Step', 'Go_Live_Planned'],
      dtype='object')

In [ ]:

Flux_Scaled['Max_Yearly_Ton_Km'] = Flux_Scaled['Max_Yearly_Tons'] * Flux_Scaled['Distance_Direct']


In [ ]:
import numpy as np

# Conditions
conditions = [
    (Flux_Scaled['Distance_Direct'] < 50),
    (Flux_Scaled['Distance_Direct'] >= 50) & (Flux_Scaled['Distance_Direct'] < 150),
    (Flux_Scaled['Distance_Direct'] >= 150) & (Flux_Scaled['Distance_Direct'] < 500),
    (Flux_Scaled['Distance_Direct'] >= 500) & (Flux_Scaled['Distance_Direct'] < 1000),
    (Flux_Scaled['Distance_Direct'] >= 1000) & (Flux_Scaled['Distance_Direct'] < 2000),
    (Flux_Scaled['Distance_Direct'] >= 2000) & (Flux_Scaled['Distance_Direct'] < 6000),
    (Flux_Scaled['Distance_Direct'] >= 6000)
]

# Valeurs correspondantes
valeurs = ["KM_LT50", "KM50-149", "KM150-499", "KM500-999", "KM1000-1999", "KM2000-5999", "KM_GE6000"]

# Appliquez les conditions et assignez les valeurs
Flux_Scaled['Category_Distance'] = np.select(conditions, valeurs, default=None)


In [ ]:
import pandas as pd

# Spécifiez le chemin complet du fichier Excel
chemin_fichier_excel = "/content/drive/MyDrive/ECTN_Routing/Eligibility_Type_Goods.xlsx"

# Nom de la feuille à lire
nom_feuille = "ECTN_Types"

# Lisez la feuille spécifique du fichier Excel
Type_Goods_Data = pd.read_excel(chemin_fichier_excel, sheet_name=nom_feuille, header=0)


In [ ]:
Type_Goods_Data.columns

Index(['Distance', 'Country', '%Eligibility', 'TOTAL', 'GT01', 'GT02', 'GT03',
       'GT04', 'GT05', 'GT06', 'GT07', 'GT08', 'GT09', 'GT10', 'GT11', 'GT12',
       'GT13', 'GT14', 'GT15', 'GT16', 'GT17', 'GT18', 'GT19', 'GT20'],
      dtype='object')

In [ ]:
#On ne garde que les trois premières colonnes
Type_Goods_Data = Type_Goods_Data[['Distance', 'Country', '%Eligibility']]

In [ ]:


# Créez une fonction pour obtenir le %Type_Goods
def calculate_type_goods(row, data):
    id_origin = row["ID_Node_Scaled_Origin"]
    country_origin = Nodes_Data.loc[Nodes_Data['ID_Node_Scaled'] == id_origin, 'Country_Scaled'].values[0]
    id_destination = row["ID_Node_Scaled_Destination"]
    country_destination = Nodes_Data.loc[Nodes_Data['ID_Node_Scaled'] == id_destination, 'Country_Scaled'].values[0]
    distance_category = row['Category_Distance']

    # Trouvez la ligne correspondant à Country_Origin et Distance_Category
    origin_row = data[(data['Country'] == country_origin) & (data['Distance'] == distance_category)]

    # Trouvez la ligne correspondant à Country_Destination et Distance_Category
    destination_row = data[(data['Country'] == country_destination) & (data['Distance'] == distance_category)]

    # Si les deux lignes existent, prenez le minimum des Eligibility
    if not origin_row.empty and not destination_row.empty:
        return min(origin_row.iloc[0]['%Eligibility'], destination_row.iloc[0]['%Eligibility'])
    else:
        return 0.32

# Appliquez la fonction pour créer la colonne %Type_Goods
Flux_Scaled['%Type_Goods'] = Flux_Scaled.apply(lambda row: calculate_type_goods(row,Type_Goods_Data), axis=1)


In [ ]:
#On créé les colonnes Eligible_Yearly_Trips,Eligible_Yearly_Tons et Eligible_Yearly_TonKm
# Assurez-vous que les colonnes %Eligibility, Max_Yearly_Trips et Max_Yeraly_Tons existent déjà dans Flux_Scaled.

# Créez les colonnes Eligible_Yearly_Trips et Eligible_Yearly_Tons en les calculant à partir des colonnes existantes
Flux_Scaled['Eligible_Yearly_Trips'] = Flux_Scaled.apply(lambda row: row['Max_Yearly_Trips'] * row['%Type_Goods'], axis=1)
Flux_Scaled['Eligible_Yearly_Tons'] = Flux_Scaled.apply(lambda row: row['Max_Yearly_Tons'] * row['%Type_Goods'], axis=1)
Flux_Scaled['Eligible_Yearly_Ton_Km'] = Flux_Scaled['Eligible_Yearly_Tons'] * Flux_Scaled['Distance_Direct']
# Affichez le DataFrame Flux_Scaled mis à jour


In [ ]:
filtered_rows = Flux_Scaled[Flux_Scaled['Eligible_Yearly_Trips'].isna() | (Flux_Scaled['Eligible_Yearly_Trips'] == 'erreur')]
filtered_rows

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,Distance_moyenne_segment,Nbe_Terminals,Step,Go_Live_Planned,Max_Yearly_Ton_Km,Category_Distance,%Type_Goods,Eligible_Yearly_Trips,Eligible_Yearly_Tons,Eligible_Yearly_Ton_Km


## Différencier Flux I/O et InterHub

In [ ]:
Terminaux_Data.columns

Index(['Terminal_Name', 'Terminal_Lat', 'Terminal_Long', 'Step',
       'Go Live Planned', 'Node_Lat', 'Node_Long', 'Transit',
       'Network_Node_ID', 'Country', 'List_Nodes_Hub'],
      dtype='object')

In [ ]:
#On créé les colonnes Trucks_I/O pour comptabiliser les flux entrants/sortants par terminaux et Trucks_TR pour les flux interhub
Terminaux_Data['Trucks_I/O'] = 0
Terminaux_Data['Trucks_TR'] = 0

In [ ]:
def update_terminal_traffic(row):
    routing = row['Routing_Final']
    traffic_flow = row['Eligible_Yearly_Trips']

    # Split du routing en terminaux
    terminals = routing.split('<>')

    if len(terminals) < 2:
        return

    first_terminal, last_terminal = terminals[0], terminals[-1]

    # Recherche des noms de terminaux correspondants dans Terminaux_Data
    first_terminal_row = Terminaux_Data[Terminaux_Data['Terminal_Name'] == first_terminal]
    last_terminal_row = Terminaux_Data[Terminaux_Data['Terminal_Name'] == last_terminal]

    if not first_terminal_row.empty:
        # Ajout de la valeur de Traffic_flow_trucks_2019 à la colonne Trucks_I/O du premier terminal
        first_terminal_index = first_terminal_row.index[0]
        Terminaux_Data.at[first_terminal_index, 'Trucks_I/O'] += traffic_flow

    if not last_terminal_row.empty:
        # Ajout de la valeur de Traffic_flow_trucks_2019 à la colonne Trucks_I/O du dernier terminal
        last_terminal_index = last_terminal_row.index[0]
        Terminaux_Data.at[last_terminal_index, 'Trucks_I/O'] += traffic_flow

    # Termini intermédiaires pour lesquels nous ajouterons Traffic_flow_trucks_2019 à la colonne Trucks_TR
    intermediate_terminals = terminals[1:-1]

    for intermediate_terminal in intermediate_terminals:
        intermediate_terminal_row = Terminaux_Data[Terminaux_Data['Terminal_Name'] == intermediate_terminal]
        if not intermediate_terminal_row.empty:
            # Ajout de la valeur de Traffic_flow_trucks_2019 à la colonne Trucks_TR pour les terminaux intermédiaires
            intermediate_terminal_index = intermediate_terminal_row.index[0]
            Terminaux_Data.at[intermediate_terminal_index, 'Trucks_TR'] += traffic_flow

# Appliquez la fonction à chaque ligne de final_merged_data
Flux_Scaled.apply(update_terminal_traffic, axis=1)


832     None
869     None
4434    None
4448    None
4532    None
        ... 
131     None
1146    None
5372    None
3750    None
3050    None
Length: 6789, dtype: object

In [ ]:

# Créer la colonne 'Trucks_Total'
Terminaux_Data['Trucks_Total'] = Terminaux_Data['Trucks_I/O'] + Terminaux_Data['Trucks_TR']



In [ ]:
Terminaux_Data.head()

,Terminal_Name,Terminal_Lat,Terminal_Long,Step,Go Live Planned,Node_Lat,Node_Long,Transit,Network_Node_ID,Country,List_Nodes_Hub,Trucks_I/O,Trucks_TR,Trucks_Total
0,Aarhus,56.149628,10.213405,S3_Ph2,2036.0,56.135541,10.156370,N,116479,DK,"[262458, 250470, 116447, 130106, 130103, 12437...",171229.059726,0.000000,171229.059726
1,Albacete - San Clemente,39.403889,-2.429444,S1_Ph3,2030.0,39.351198,-2.441125,Y,250086,ES,"[193560, 193559, 250086, 191616, 101311, 10132...",3869.083114,28582.897579,32451.980692
2,Algésiras,36.127500,-5.453889,S1_Ph3,2030.0,36.112912,-5.455857,N,101665,ES,"[101772, 101658, 101659, 101766, 124426, 10176...",13589.843483,0.000000,13589.843483
3,Alicante - Crevillent,38.248611,-0.808889,S1_Ph3,2030.0,38.220704,-0.812785,Y,101570,ES,"[193423, 101815, 101555, 101564, 101814, 10182...",59852.104962,49893.695449,109745.800411
4,Almería,36.840000,-2.467500,S1_Ph3,2030.0,36.838037,-2.447375,N,101645,ES,"[250093, 101653, 101705, 101708, 101656, 10170...",13420.102305,0.000000,13420.102305


# Calcul Différence Distance

In [ ]:
Flux_Scaled['Distance_InterHub_Final']

832         [321.204, 304.084, 206.608, 257.183, 256.451]
869                  [307.838, 316.677, 240.296, 318.326]
4434                  [257.791, 345.539, 281.48, 155.262]
4448                  [316.526, 253.06, 336.442, 264.553]
4532                          [258.001, 252.053, 346.615]
                              ...                        
131     [255.235, 158.891, 225.359, 241.965, 262.892, ...
1146    [391.287, 373.673, 244.519, 188.054, 230.59, 1...
5372    [263.054, 307.566, 257.791, 131.127, 314.855, ...
3750                           [174.11, 221.982, 314.855]
3050                          [314.855, 229.921, 151.844]
Name: Distance_InterHub_Final, Length: 6789, dtype: object

In [ ]:

# Créez la colonne Distance_ECTN en calculant la somme des valeurs de la liste Distance_InterHub_Final
Flux_Scaled['Distance_ECTN'] = Flux_Scaled['Distance_InterHub_Final'].apply(lambda x: sum(x))

# Supposons que vous ayez déjà la colonne Distance_Direct
# Créez la colonne Ecart_relatif_distance en appliquant la formule
Flux_Scaled['Ecart_relatif_distance'] = ((Flux_Scaled['Distance_ECTN'] - Flux_Scaled['Distance_Direct']) / Flux_Scaled['Distance_Direct'])

# Arrondissez la colonne Ecart_relatif_distance à deux décimales
Flux_Scaled['Ecart_relatif_distance'] = Flux_Scaled['Ecart_relatif_distance'].round(2)



# Autres

In [ ]:
# Renommer les colonnes
NUTS3_Data = NUTS3_Data.rename(columns={
    'Country': 'Country_Original',
    'Geometric_center': 'Geometric_center_Original',
    'Name_Nuts3_Scaled': 'Name_Hub_Scaled'
})

# Réorganiser les colonnes
NUTS3_Data = NUTS3_Data[[
    'ID_Nuts3_original', 'Name_Nuts3_Original', 'Country_Original', 'ID_Node_Original',
    'Geometric_center_Original', 'Geometric_center_X_Original', 'Geometric_center_Y_Original',
    'ID_NUTS3_Scaled', 'Name_Hub_Scaled', 'Name_City_Scaled', 'ID_Node_Scaled'
]]


In [ ]:
NUTS3_Data_Original = pd.read_csv("drive/MyDrive/ECTN_Routing/02_NUTS-3-Regions.csv")

In [ ]:
# Importer les colonnes pertinentes de NUTS3_Data_Original dans NUTS3_Data
NUTS3_Data['Country_Scaled'] = NUTS3_Data['ID_NUTS3_Scaled'].map(NUTS3_Data_Original.set_index('ETISPlus_Zone_ID')['Country'])
NUTS3_Data['Name_Nuts3_Scaled'] = NUTS3_Data['ID_NUTS3_Scaled'].map(NUTS3_Data_Original.set_index('ETISPlus_Zone_ID')['Name'])


In [ ]:
nombre_de_lignes = len(NUTS3_Data)
print("Nombre de lignes dans NUTS3_Data :", nombre_de_lignes)


Nombre de lignes dans NUTS3_Data : 1123


In [ ]:
Terminaux_Data.head()

,Terminal_Name,Terminal_Lat,Terminal_Long,Step,Go Live Planned,Node_Lat,Node_Long,Transit,Network_Node_ID,Country,List_Nodes_Hub,Trucks_I/O,Trucks_TR,Trucks_Total
0,Aarhus,56.149628,10.213405,S3_Ph2,2036.0,56.135541,10.156370,N,116479,DK,"[262458, 250470, 116447, 130106, 130103, 12437...",171229.059726,0.000000,171229.059726
1,Albacete - San Clemente,39.403889,-2.429444,S1_Ph3,2030.0,39.351198,-2.441125,Y,250086,ES,"[193560, 193559, 250086, 191616, 101311, 10132...",3869.083114,28582.897579,32451.980692
2,Algésiras,36.127500,-5.453889,S1_Ph3,2030.0,36.112912,-5.455857,N,101665,ES,"[101772, 101658, 101659, 101766, 124426, 10176...",13589.843483,0.000000,13589.843483
3,Alicante - Crevillent,38.248611,-0.808889,S1_Ph3,2030.0,38.220704,-0.812785,Y,101570,ES,"[193423, 101815, 101555, 101564, 101814, 10182...",59852.104962,49893.695449,109745.800411
4,Almería,36.840000,-2.467500,S1_Ph3,2030.0,36.838037,-2.447375,N,101645,ES,"[250093, 101653, 101705, 101708, 101656, 10170...",13420.102305,0.000000,13420.102305


In [ ]:
import numpy as np

# Créez la colonne 'Terminal_Type' en appliquant les conditions avec égal à 0.0
Terminaux_Data['Terminal_Type'] = np.where(
    (Terminaux_Data['Trucks_TR'] == 0.0) & (Terminaux_Data['Trucks_I/O'] != 0.0), 'Satellite',
    np.where(
        (Terminaux_Data['Trucks_TR'] != 0.0) & (Terminaux_Data['Trucks_I/O'] != 0.0), 'Main',
        np.where(
            (Terminaux_Data['Trucks_TR'] == 0.0) & (Terminaux_Data['Trucks_I/O'] == 0.0), np.nan, 'Transit'
        )
    )
)


In [ ]:
Flux_Scaled.columns

Index(['Corridor_Générique', 'Max_Yearly_Tons', 'Max_Yearly_Trips',
       'Distance_Direct_ETIS_plus', 'ID_Node_Scaled_Origin',
       'ID_Node_Scaled_Destination', 'Edge_Path', 'Nodes_Path', 'Routing',
       'Distance_Direct', 'Routing_Bis1', 'Routing_Bis2', 'Routing_Bis3',
       'Distance_InterHub', 'Routing_Final', 'Distance_InterHub_Final', '<200',
       '[200,300]', '[300,350]', '>350', 'Distance_moyenne_segment',
       'Nbe_Terminals', 'Step', 'Go_Live_Planned', 'Max_Yearly_Ton_Km',
       'Category_Distance', '%Type_Goods', 'Eligible_Yearly_Trips',
       'Eligible_Yearly_Tons', 'Eligible_Yearly_Ton_Km', 'Distance_ECTN',
       'Ecart_relatif_distance'],
      dtype='object')

In [ ]:
!pip install XlsxWriter


# Flux_Non_Captés

# Nouvelle section

In [ ]:
#On veut voir les gros corridors qu'on a pas encore captés
Flux_Data_Full = pd.read_excel("drive/MyDrive/ECTN_Routing/Corridor_97%.xlsx", sheet_name=2, header=0)

In [ ]:
Flux_Data_Full.columns

Index(['ID_origin_NUTS3', 'Name_origin_NUTS3', 'ID_destination_NUTS3',
       'Name_destination_NUTS3', 'Edge_path_E_road',
       'Distance_from_origin_region_to_E_road', 'Distance_within_E_road',
       'Distance_from_E_road_to_destination_region', 'Total_distance_Mendeley',
       'Traffic_flow_trucks_2019', 'Traffic_flow_tons_2019',
       'Corridor_générique', 'Total_Distance_PTV', 'Eligibilité '],
      dtype='object')

In [ ]:
#Ne contient que les flux *intégrés avec le corridor directionnel scaled.
Flux_Integrated= Flux_Data

In [ ]:
Flux_Integrated.columns

Index(['ID_origin_NUTS3', 'Name_origin_NUTS3', 'ID_destination_NUTS3',
       'Name_destination_NUTS3', 'Edge_path_E_road',
       'Distance_from_origin_region_to_E_road', 'Distance_within_E_road',
       'Distance_from_E_road_to_destination_region', 'Total_distance_Mendeley',
       'Traffic_flow_trucks_2019', 'Traffic_flow_tons_2019',
       'Corridor_générique', 'Total_Distance_PTV', 'Eligibilité ',
       'ID_Node_Original_Origin', 'Name_Scaled_Origin',
       'ID_Node_Scaled_Origin', 'ID_NUTS3_Scaled_Origin',
       'ID_Node_Original_Destination', 'Name_Scaled_Destination',
       'ID_Node_Scaled_Destination', 'ID_NUTS3_Scaled_Destination',
       'corridor_directionnel'],
      dtype='object')

In [ ]:
Flux_Integrated['corridor_directionnel_bis'] = Flux_Integrated.apply(lambda row: f"{row['Name_origin_NUTS3']} > {row['Name_destination_NUTS3']}", axis=1)

# Afficher le DataFrame avec la nouvelle colonne
Flux_Integrated.columns

Index(['ID_origin_NUTS3', 'Name_origin_NUTS3', 'ID_destination_NUTS3',
       'Name_destination_NUTS3', 'Edge_path_E_road',
       'Distance_from_origin_region_to_E_road', 'Distance_within_E_road',
       'Distance_from_E_road_to_destination_region', 'Total_distance_Mendeley',
       'Traffic_flow_trucks_2019', 'Traffic_flow_tons_2019',
       'Corridor_générique', 'Total_Distance_PTV', 'Eligibilité ',
       'ID_Node_Original_Origin', 'Name_Scaled_Origin',
       'ID_Node_Scaled_Origin', 'ID_NUTS3_Scaled_Origin',
       'ID_Node_Original_Destination', 'Name_Scaled_Destination',
       'ID_Node_Scaled_Destination', 'ID_NUTS3_Scaled_Destination',
       'corridor_directionnel', 'corridor_directionnel_bis'],
      dtype='object')

In [ ]:
# Fonction pour créer le corridor générique en ordre alphabétique
def create_generic_corridor(row):
    # Séparez le corridor en deux parties en utilisant le caractère ">"
    parts = row['corridor_directionnel_bis'].split(' > ')

    # Triez les parties pour obtenir un ordre standardisé (A > B)
    parts.sort()

    # Rejoignez les parties pour créer le corridor générique
    return ' <> '.join(parts)

# Appliquez la fonction pour créer la colonne "Corridor Générique" dans Flux_Data_Directionnel
Flux_Integrated['Corridor_Générique'] = Flux_Integrated.apply(create_generic_corridor, axis=1)

# Triez le dataframe Flux_Data_Directionnel en utilisant la colonne "Corridor Générique"
Flux_Integrated.sort_values(by='Corridor_Générique', inplace=True)

# Réinitialisez l'index si nécessaire
Flux_Integrated.reset_index(drop=True, inplace=True)

In [ ]:
valeurs_uniques = Flux_Integrated['Corridor_Générique'].unique()
len(valeurs_uniques)

33196

In [ ]:
valeurs_uniques_bis = Flux_Integrated['corridor_directionnel_bis'].unique()
len(valeurs_uniques_bis)

65924

In [ ]:

# Calcul de la somme des valeurs de la colonne "Max_Yearly_Trips"
somme_max_yearly_trips = Flux_Integrated['Traffic_flow_trucks_2019'].sum()*2

# Affichage de la somme
print("Somme des valeurs de Max_Yearly_Trips :", somme_max_yearly_trips)

Somme des valeurs de Max_Yearly_Trips : 49235980.0


In [ ]:
Flux_Data_Full['corridor_directionnel'] = Flux_Data_Full.apply(lambda row: f"{row['Name_origin_NUTS3']} > {row['Name_destination_NUTS3']}", axis=1)
Flux_Data_Directionnel_Full = Flux_Data_Full.groupby('corridor_directionnel').agg({
    'Traffic_flow_trucks_2019': 'sum',
    'Traffic_flow_tons_2019': 'sum',
    'Total_Distance_PTV': 'mean',
    'ID_origin_NUTS3': 'mean',
    'ID_destination_NUTS3': 'mean',
}).reset_index()

In [ ]:
import pandas as pd

# Créer la colonne 'Integration' en fonction de la présence dans la liste des valeurs uniques
Flux_Data_Directionnel_Full['Integration_bis'] = Flux_Data_Directionnel_Full['corridor_directionnel'].apply(lambda x: 'Integrated' if x in valeurs_uniques_bis else 'Non Integrated')

# Afficher le DataFrame avec la nouvelle colonne
Flux_Data_Directionnel_Full.head()

,corridor_directionnel,Traffic_flow_trucks_2019,Traffic_flow_tons_2019,Total_Distance_PTV,ID_origin_NUTS3,ID_destination_NUTS3,Integration_bis
0,A Coruna > Ain,231.25,3145,1569.000,110010101.0,112070101.0,Integrated
1,A Coruna > Aisne,396.25,5389,1599.379,110010101.0,112020201.0,Integrated
2,A Coruna > Alessandria,160.00,2176,1783.000,110010101.0,118120108.0,Integrated
3,A Coruna > Algarve,1246.25,16949,791.663,110010101.0,127010500.0,Non Integrated
4,A Coruna > Alicante / Alacant,2566.25,34901,947.000,110010101.0,110050201.0,Integrated


In [ ]:


# Filtrer les lignes où Integration est égal à "Integrated"
filtre_integration = Flux_Data_Directionnel_Full['Integration_bis'] == 'Integrated'
somme_max_yearly_trips_integrated = Flux_Data_Directionnel_Full.loc[filtre_integration, 'Traffic_flow_trucks_2019'].sum()*2

# Afficher la somme
print("Somme de Max_Yearly_Trips pour Integration égal à 'Integrated' :", somme_max_yearly_trips_integrated)


Somme de Max_Yearly_Trips pour Integration égal à 'Integrated' : 49235980.0


In [ ]:
# Fonction pour créer le corridor générique en ordre alphabétique
def create_generic_corridor(row):
    # Séparez le corridor en deux parties en utilisant le caractère ">"
    parts = row['corridor_directionnel'].split(' > ')

    # Triez les parties pour obtenir un ordre standardisé (A > B)
    parts.sort()

    # Rejoignez les parties pour créer le corridor générique
    return ' <> '.join(parts)

# Appliquez la fonction pour créer la colonne "Corridor Générique" dans Flux_Data_Directionnel
Flux_Data_Directionnel_Full['Corridor_Générique'] = Flux_Data_Directionnel_Full.apply(create_generic_corridor, axis=1)

# Triez le dataframe Flux_Data_Directionnel en utilisant la colonne "Corridor Générique"
Flux_Data_Directionnel_Full.sort_values(by='Corridor_Générique', inplace=True)

# Réinitialisez l'index si nécessaire
Flux_Data_Directionnel_Full.reset_index(drop=True, inplace=True)

In [ ]:
Flux_Data_Générique_Full = Flux_Data_Directionnel_Full.groupby('Corridor_Générique').agg({
    'Traffic_flow_trucks_2019': 'min',
    'Traffic_flow_tons_2019': 'min',
    'Total_Distance_PTV': 'mean',
    'ID_origin_NUTS3': 'min',
    'ID_destination_NUTS3': 'max',
}).reset_index()

In [ ]:
import pandas as pd

# Créer la colonne 'Integration' en fonction de la présence dans la liste des valeurs uniques
Flux_Data_Générique_Full['Integration'] = Flux_Data_Générique_Full['Corridor_Générique'].apply(lambda x: 'Integrated' if x in valeurs_uniques else 'Non Integrated')

# Afficher le DataFrame avec la nouvelle colonne
Flux_Data_Générique_Full.head()


,Corridor_Générique,Traffic_flow_trucks_2019,Traffic_flow_tons_2019,Total_Distance_PTV,ID_origin_NUTS3,ID_destination_NUTS3,Integration
0,A Coruna <> Ain,137.50,1870,1569.000,110010101.0,112070101.0,Integrated
1,A Coruna <> Aisne,228.75,3111,1599.379,110010101.0,112020201.0,Integrated
2,A Coruna <> Alessandria,87.50,1190,1783.000,110010101.0,118120108.0,Integrated
3,A Coruna <> Algarve,291.25,3961,791.663,110010101.0,127010500.0,Non Integrated
4,A Coruna <> Alicante / Alacant,126.25,1717,947.000,110010101.0,110050201.0,Integrated


In [ ]:
filtre_integration = Flux_Data_Générique_Full[Flux_Data_Générique_Full['Integration'] == 'Integrated']

len(filtre_integration['Corridor_Générique'].unique())

33196

In [ ]:


# Filtrer les lignes où Integration est égal à "Integrated"
filtre_integration = Flux_Data_Générique_Full['Integration'] == 'Integrated'
somme_max_yearly_trips_integrated = Flux_Data_Générique_Full.loc[filtre_integration, 'Traffic_flow_trucks_2019'].sum()*2

# Afficher la somme
print("Somme de Max_Yearly_Trips pour Integration égal à 'Integrated' :", somme_max_yearly_trips_integrated)


Somme de Max_Yearly_Trips pour Integration égal à 'Integrated' : 18071847.5


In [ ]:
# Renommez les colonnes
Flux_Data_Générique_Full.rename(columns={'Traffic_flow_tons_2019': 'Max_Yearly_Tons', 'Traffic_flow_trucks_2019': 'Max_Yearly_Trips'}, inplace=True)

# Multipliez les valeurs par deux
Flux_Data_Générique_Full['Max_Yearly_Tons'] *= 2
Flux_Data_Générique_Full['Max_Yearly_Trips'] *= 2

In [ ]:
# Convertir les colonnes ID_origin_NUTS3 et ID_destination_NUTS3 en entiers
Flux_Data_Générique_Full['ID_origin_NUTS3'] = Flux_Data_Générique_Full['ID_origin_NUTS3'].astype(int)
Flux_Data_Générique_Full['ID_destination_NUTS3'] = Flux_Data_Générique_Full['ID_destination_NUTS3'].astype(int)


In [ ]:
Flux_Data_Générique_Full.columns

Index(['Corridor_Générique', 'Max_Yearly_Trips', 'Max_Yearly_Tons',
       'Total_Distance_PTV', 'ID_origin_NUTS3', 'ID_destination_NUTS3',
       'Integration'],
      dtype='object')

In [ ]:
nombre_de_lignes = len(Flux_Integrated)
print("Nombre de lignes dans Flux_Integrated :", nombre_de_lignes)


Nombre de lignes dans Flux_Integrated : 65970


In [ ]:


# Calcul de la somme des valeurs de la colonne "Max_Yearly_Trips"
somme_max_yearly_trips = Flux_Scaled['Max_Yearly_Trips'].sum()

# Affichage de la somme
print("Somme des valeurs de Max_Yearly_Trips :", somme_max_yearly_trips)


Somme des valeurs de Max_Yearly_Trips : 19743362.5


In [ ]:
Flux_Integrated.columns

Index(['ID_origin_NUTS3', 'Name_origin_NUTS3', 'ID_destination_NUTS3',
       'Name_destination_NUTS3', 'Edge_path_E_road',
       'Distance_from_origin_region_to_E_road', 'Distance_within_E_road',
       'Distance_from_E_road_to_destination_region', 'Total_distance_Mendeley',
       'Traffic_flow_trucks_2019', 'Traffic_flow_tons_2019',
       'Corridor_générique', 'Total_Distance_PTV', 'Eligibilité ',
       'ID_Node_Original_Origin', 'Name_Scaled_Origin',
       'ID_Node_Scaled_Origin', 'ID_NUTS3_Scaled_Origin',
       'ID_Node_Original_Destination', 'Name_Scaled_Destination',
       'ID_Node_Scaled_Destination', 'ID_NUTS3_Scaled_Destination',
       'corridor_directionnel', 'corridor_directionnel_bis',
       'Corridor_Générique'],
      dtype='object')

In [ ]:
# Compter le nombre de lignes avec "Integration" égal à "integrated"
count_integrated = Flux_Data_Générique_Full['Integration'].value_counts().get("Integrated", 0)

print("Nombre de lignes 'Integrated' dans Flux_Data_Générique_Full :", count_integrated)


Nombre de lignes 'Integrated' dans Flux_Data_Générique_Full : 33196


In [ ]:
# Créez la colonne 'Max_Yearly_TonKm'
Flux_Data_Générique_Full['Max_Yearly_TonKm'] = Flux_Data_Générique_Full['Max_Yearly_Tons'] * Flux_Data_Générique_Full['Total_Distance_PTV']

# Réorganisez les colonnes pour placer 'Max_Yearly_TonKm' à droite de 'Max_Yearly_Tons'
Flux_Data_Générique_Full = Flux_Data_Générique_Full[['Corridor_Générique','Max_Yearly_Trips', 'Max_Yearly_Tons', 'Max_Yearly_TonKm',
                                                       'Total_Distance_PTV', 'ID_origin_NUTS3', 'ID_destination_NUTS3',
                                                       'Integration']]


In [ ]:
Flux_Data_Générique_Full = Flux_Data_Générique_Full.sort_values(by='Max_Yearly_TonKm', ascending=False)
Flux_Data_Générique_Full = Flux_Data_Générique_Full.reset_index(drop=True)

In [ ]:
Flux_Data_Générique_Full.head()

,Corridor_Générique,Max_Yearly_Trips,Max_Yearly_Tons,Max_Yearly_TonKm,Total_Distance_PTV,ID_origin_NUTS3,ID_destination_NUTS3,Integration
0,Barcelona <> Nord,67365.0,916164,1.166517e+09,1273.262,110050101,112030001,Integrated
1,Barcelona <> Pas-de-Calais,41205.0,560388,7.229039e+08,1290.006,110050101,112030002,Integrated
2,Barcelona <> Moselle,33317.5,453118,5.183761e+08,1144.020,110050101,112040103,Integrated
3,Barcelona <> Pontevedra,29132.5,396202,4.562500e+08,1151.559,110010104,110050101,Integrated
4,Barcelona <> Sevilla,30932.5,420682,4.342999e+08,1032.371,110050101,110060108,Integrated


In [ ]:
Flux_Data_Générique_Full.head(15)


,Corridor_Générique,Max_Yearly_Trips,Max_Yearly_Tons,Max_Yearly_TonKm,Total_Distance_PTV,ID_origin_NUTS3,ID_destination_NUTS3,Integration
0,Barcelona <> Nord,67365.0,916164,1.166517e+09,1273.262,110050101,112030001,Integrated
1,Barcelona <> Pas-de-Calais,41205.0,560388,7.229039e+08,1290.006,110050101,112030002,Integrated
2,Barcelona <> Moselle,33317.5,453118,5.183761e+08,1144.020,110050101,112040103,Integrated
3,Barcelona <> Pontevedra,29132.5,396202,4.562500e+08,1151.559,110010104,110050101,Integrated
4,Barcelona <> Sevilla,30932.5,420682,4.342999e+08,1032.371,110050101,110060108,Integrated
5,Barcelona <> Bas-Rhin,23130.0,314568,3.623757e+08,1151.979,110050101,112040201,Integrated
6,Madrid <> Nord,17270.0,234872,3.509659e+08,1494.286,110030000,112030001,Integrated
7,Barcelona <> Haut-Rhin,24360.0,331296,3.482110e+08,1051.057,110050101,112040202,Integrated
8,A Coruna <> Sevilla,25042.5,340578,3.372420e+08,990.205,110010101,110060108,Integrated
9,Barcelona <> Grande Porto,20820.0,283152,3.243942e+08,1145.654,110050101,127010104,Integrated


In [ ]:
#On import la table Originale avce tous les NUTS3
NUTS3_Data2 = pd.read_csv("drive/MyDrive/ECTN_Routing/02_NUTS-3-Regions.csv")
# Merge entre Flux_Data_Générique_Full et NUTS3_Data2 pour 'ID_origin_NUTS3'
Flux_Data_Générique_Full = Flux_Data_Générique_Full.merge(
    NUTS3_Data2[['ETISPlus_Zone_ID', 'Country', 'Network_Node_ID']],
    left_on='ID_origin_NUTS3',
    right_on='ETISPlus_Zone_ID',
    how='left'
)

# Renommer les colonnes résultantes
Flux_Data_Générique_Full = Flux_Data_Générique_Full.rename(columns={
    'Country': 'Country_Origin',
    'Network_Node_ID': 'Network_Node_ID_Origin'
})

# Supprimer la colonne 'ETISPlus_Zone_ID' si nécessaire
Flux_Data_Générique_Full = Flux_Data_Générique_Full.drop(columns='ETISPlus_Zone_ID')

# Merge entre Flux_Data_Générique_Full et NUTS3_Data2 pour 'ID_destination_NUTS3'
Flux_Data_Générique_Full = Flux_Data_Générique_Full.merge(
    NUTS3_Data2[['ETISPlus_Zone_ID', 'Country', 'Network_Node_ID']],
    left_on='ID_destination_NUTS3',
    right_on='ETISPlus_Zone_ID',
    how='left'
)

# Renommer les colonnes résultantes
Flux_Data_Générique_Full = Flux_Data_Générique_Full.rename(columns={
    'Country': 'Country_Destination',
    'Network_Node_ID': 'Network_Node_ID_Destination'
})

# Supprimer la colonne 'ETISPlus_Zone_ID' si nécessaire
Flux_Data_Générique_Full = Flux_Data_Générique_Full.drop(columns='ETISPlus_Zone_ID')


In [ ]:
#On créé les corridor générique Country.
# Convertissez les colonnes Country_Origin et Country_Destination en chaînes de caractères
Flux_Data_Générique_Full['Country_Origin'] = Flux_Data_Générique_Full['Country_Origin'].astype(str)
Flux_Data_Générique_Full['Country_Destination'] = Flux_Data_Générique_Full['Country_Destination'].astype(str)

# Créez une fonction pour trier et concaténer les pays
def sort_and_concatenate_countries(row):
    countries = [row['Country_Origin'], row['Country_Destination']]
    countries.sort()  # Triez les pays par ordre alphabétique
    return f"{countries[0]}<>{countries[1]}"

# Appliquez la fonction à chaque ligne pour créer la colonne Corridor_Country
Flux_Data_Générique_Full['Corridor_Country'] = Flux_Data_Générique_Full.apply(sort_and_concatenate_countries, axis=1)


In [ ]:
Flux_Data_Générique_Full.columns

Index(['Corridor_Générique', 'Max_Yearly_Trips', 'Max_Yearly_Tons',
       'Max_Yearly_TonKm', 'Total_Distance_PTV', 'ID_origin_NUTS3',
       'ID_destination_NUTS3', 'Integration', 'Country_Origin',
       'Network_Node_ID_Origin', 'Country_Destination',
       'Network_Node_ID_Destination', 'Corridor_Country'],
      dtype='object')

In [ ]:
import numpy as np

# Conditions
conditions = [
    (Flux_Data_Générique_Full['Total_Distance_PTV'] < 50),
    (Flux_Data_Générique_Full['Total_Distance_PTV'] >= 50) & (Flux_Data_Générique_Full['Total_Distance_PTV'] < 150),
    (Flux_Data_Générique_Full['Total_Distance_PTV'] >= 150) & (Flux_Data_Générique_Full['Total_Distance_PTV'] < 500),
    (Flux_Data_Générique_Full['Total_Distance_PTV'] >= 500) & (Flux_Data_Générique_Full['Total_Distance_PTV'] < 1000),
    (Flux_Data_Générique_Full['Total_Distance_PTV'] >= 1000) & (Flux_Data_Générique_Full['Total_Distance_PTV'] < 2000),
    (Flux_Data_Générique_Full['Total_Distance_PTV'] >= 2000) & (Flux_Data_Générique_Full['Total_Distance_PTV'] < 6000),
    (Flux_Data_Générique_Full['Total_Distance_PTV'] >= 6000)
]

# Valeurs correspondantes
valeurs = ["KM_LT50", "KM50-149", "KM150-499", "KM500-999", "KM1000-1999", "KM2000-5999", "KM_GE6000"]

# Appliquez les conditions et assignez les valeurs
Flux_Data_Générique_Full['Category_Distance'] = np.select(conditions, valeurs, default=None)

In [ ]:
Flux_Data_Générique_Full.columns

Index(['Corridor_Générique', 'Max_Yearly_Trips', 'Max_Yearly_Tons',
       'Max_Yearly_TonKm', 'Total_Distance_PTV', 'ID_origin_NUTS3',
       'ID_destination_NUTS3', 'Integration', 'Country_Origin',
       'Network_Node_ID_Origin', 'Country_Destination',
       'Network_Node_ID_Destination', 'Corridor_Country', 'Category_Distance'],
      dtype='object')

In [ ]:
# Créez une fonction pour obtenir le %Type_Goods
def calculate_type_goods(row, data):
    country_origin = row['Country_Origin']
    country_destination = row['Country_Destination']
    distance_category = row["Category_Distance"]

    # Trouvez la ligne correspondant à Country_Origin et Distance_Category
    origin_row = data[(data['Country'] == country_origin) & (data['Distance'] == distance_category)]

    # Trouvez la ligne correspondant à Country_Destination et Distance_Category
    destination_row = data[(data['Country'] == country_destination) & (data['Distance'] == distance_category)]

    # Si les deux lignes existent, prenez le minimum des Eligibility
    if not origin_row.empty and not destination_row.empty:
        return min(origin_row.iloc[0]['%Eligibility'], destination_row.iloc[0]['%Eligibility'])
    else:
        return 0.32

# Appliquez la fonction pour créer la colonne %Type_Goods
Flux_Data_Générique_Full['%Type_Goods'] = Flux_Data_Générique_Full.apply(lambda row: calculate_type_goods(row,Type_Goods_Data), axis=1)


In [ ]:
#On créé les colonnes Eligible_Yearly_Trips,Eligible_Yearly_Tons et Eligible_Yearly_TonKm
# Assurez-vous que les colonnes %Eligibility, Max_Yearly_Trips et Max_Yeraly_Tons existent déjà dans Flux_Scaled.

# Créez les colonnes Eligible_Yearly_Trips et Eligible_Yearly_Tons en les calculant à partir des colonnes existantes
Flux_Data_Générique_Full['Eligible_Yearly_Trips'] = Flux_Data_Générique_Full.apply(lambda row: row['Max_Yearly_Trips'] * row['%Type_Goods'], axis=1)
Flux_Data_Générique_Full['Eligible_Yearly_Tons'] = Flux_Data_Générique_Full.apply(lambda row: row['Max_Yearly_Tons'] * row['%Type_Goods'], axis=1)
Flux_Data_Générique_Full['Eligible_Yearly_Ton_Km'] = Flux_Data_Générique_Full['Eligible_Yearly_Tons'] * Flux_Data_Générique_Full['Total_Distance_PTV']
# Affichez le DataFrame Flux_Scaled mis à jour

In [ ]:
#On créé une table Country_Analysis qui nous permet de classer les Corridor_Country par OD de flux
# Étape 1 : Filtrer les données
filtered_data = Flux_Data_Générique_Full[Flux_Data_Générique_Full['Integration'] == 'Integrated']

# Étape 2 : Grouper par Corridor_Country
grouped_data = Flux_Data_Générique_Full.groupby('Corridor_Country')

# Étape 3 : Calculer les sommes
Flux_Country = grouped_data.agg({
    'Max_Yearly_Trips': 'sum',
    'Max_Yearly_Tons': 'sum',
    'Max_Yearly_TonKm' :'sum',
    'Eligible_Yearly_Trips': 'sum',
    'Eligible_Yearly_Tons': 'sum',
    'Eligible_Yearly_Ton_Km' :'sum'

}).reset_index()

# Renommer les colonnes
Flux_Country.rename(columns={'Max_Yearly_Trips': 'Max_Yearly_Trips_Tot', 'Max_Yearly_Tons': 'Max_Yearly_Tons_Tot','Max_Yearly_TonKm': 'Max_Yearly_TonKm_Tot'}, inplace=True)

# Renommer les colonnes
Flux_Country.rename(columns={'Eligible_Yearly_Trips': 'Eligible_Yearly_Trips_Tot', 'Eligible_Yearly_Tons': 'Eligible_Yearly_Tons_Tot','Eligible_Yearly_Ton_Km': 'Eligible_Yearly_Ton_Km_Tot'}, inplace=True)

In [ ]:
Flux_Data_Générique_Full.columns

Index(['Corridor_Générique', 'Max_Yearly_Trips', 'Max_Yearly_Tons',
       'Max_Yearly_TonKm', 'Total_Distance_PTV', 'ID_origin_NUTS3',
       'ID_destination_NUTS3', 'Integration', 'Country_Origin',
       'Network_Node_ID_Origin', 'Country_Destination',
       'Network_Node_ID_Destination', 'Corridor_Country', 'Category_Distance',
       '%Type_Goods', 'Eligible_Yearly_Trips', 'Eligible_Yearly_Tons',
       'Eligible_Yearly_Ton_Km'],
      dtype='object')

In [ ]:
# Créer une colonne Max_Yearly_Trips_Planned avec des valeurs initiales à 0
Flux_Country['Max_Yearly_Trips_Planned'] = 0

# Boucle pour parcourir les valeurs uniques de Corridor_Country
for corridor_country in Flux_Country['Corridor_Country'].unique():
    # Filtrer les données de Flux_Data_Générique_Full pour la valeur de Corridor_Country et Integration=='Integrated'
    filtered_data = Flux_Data_Générique_Full[(Flux_Data_Générique_Full['Corridor_Country'] == corridor_country) & (Flux_Data_Générique_Full['Integration'] == 'Integrated')]

    # Calculer la somme de Max_Yearly_Trips pour les données filtrées
    max_yearly_trips_planned = filtered_data['Max_Yearly_Trips'].sum()

    # Mettre à jour la valeur correspondante dans Flux_Country
    Flux_Country.loc[Flux_Country['Corridor_Country'] == corridor_country, 'Max_Yearly_Trips_Planned'] = max_yearly_trips_planned
    Flux_Country['%Max_Yearly_Trips_Planned'] = Flux_Country['Max_Yearly_Trips_Planned']/Flux_Country['Max_Yearly_Trips_Tot']*100

In [ ]:
# Créer une colonne Max_Yearly_Trips_Planned avec des valeurs initiales à 0
Flux_Country['Eligible_Yearly_Trips_Planned'] = 0

# Boucle pour parcourir les valeurs uniques de Corridor_Country
for corridor_country in Flux_Country['Corridor_Country'].unique():
    # Filtrer les données de Flux_Data_Générique_Full pour la valeur de Corridor_Country et Integration=='Integrated'
    filtered_data = Flux_Data_Générique_Full[(Flux_Data_Générique_Full['Corridor_Country'] == corridor_country) & (Flux_Data_Générique_Full['Integration'] == 'Integrated')]

    # Calculer la somme de Max_Yearly_Trips pour les données filtrées
    max_yearly_trips_planned = filtered_data['Eligible_Yearly_Trips'].sum()

    # Mettre à jour la valeur correspondante dans Flux_Country
    Flux_Country.loc[Flux_Country['Corridor_Country'] == corridor_country, 'Eligible_Yearly_Trips_Planned'] = max_yearly_trips_planned
    Flux_Country['%Eligible_Yearly_Trips_Planned'] = Flux_Country['Eligible_Yearly_Trips_Planned']/Flux_Country['Eligible_Yearly_Trips_Tot']*100

In [ ]:
# Créer une colonne Max_Yearly_Trips_Planned avec des valeurs initiales à 0
Flux_Country['Max_Yearly_TonKm_Planned'] = 0

# Boucle pour parcourir les valeurs uniques de Corridor_Country
for corridor_country in Flux_Country['Corridor_Country'].unique():
    # Filtrer les données de Flux_Data_Générique_Full pour la valeur de Corridor_Country et Integration=='Integrated'
    filtered_data = Flux_Data_Générique_Full[(Flux_Data_Générique_Full['Corridor_Country'] == corridor_country) & (Flux_Data_Générique_Full['Integration'] == 'Integrated')]

    # Calculer la somme de Max_Yearly_Trips pour les données filtrées
    max_yearly_tons_planned = filtered_data['Max_Yearly_TonKm'].sum()

    # Mettre à jour la valeur correspondante dans Flux_Country
    Flux_Country.loc[Flux_Country['Corridor_Country'] == corridor_country, 'Max_Yearly_TonKm_Planned'] = max_yearly_tons_planned
    Flux_Country['%Max_Yearly_TonKm_Planned'] = Flux_Country['Max_Yearly_TonKm_Planned']/Flux_Country['Max_Yearly_TonKm_Tot']*100

In [ ]:
# Créer une colonne Max_Yearly_Trips_Planned avec des valeurs initiales à 0
Flux_Country['Eligible_Yearly_Ton_Km_Planned'] = 0

# Boucle pour parcourir les valeurs uniques de Corridor_Country
for corridor_country in Flux_Country['Corridor_Country'].unique():
    # Filtrer les données de Flux_Data_Générique_Full pour la valeur de Corridor_Country et Integration=='Integrated'
    filtered_data = Flux_Data_Générique_Full[(Flux_Data_Générique_Full['Corridor_Country'] == corridor_country) & (Flux_Data_Générique_Full['Integration'] == 'Integrated')]

    # Calculer la somme de Max_Yearly_Trips pour les données filtrées
    max_yearly_tons_planned = filtered_data['Eligible_Yearly_Ton_Km'].sum()

    # Mettre à jour la valeur correspondante dans Flux_Country
    Flux_Country.loc[Flux_Country['Corridor_Country'] == corridor_country, 'Eligible_Yearly_Ton_Km_Planned'] = max_yearly_tons_planned
    Flux_Country['%Eligible_Yearly_Ton_Km_Planned'] = Flux_Country['Eligible_Yearly_Ton_Km_Planned']/Flux_Country['Eligible_Yearly_Ton_Km_Tot']*100

In [ ]:
Flux_Country = Flux_Country.sort_values(by='Max_Yearly_TonKm_Tot', ascending=False)
Flux_Country.head()

,Corridor_Country,Max_Yearly_Trips_Tot,Max_Yearly_Tons_Tot,Max_Yearly_TonKm_Tot,Eligible_Yearly_Trips_Tot,Eligible_Yearly_Tons_Tot,Eligible_Yearly_Ton_Km_Tot,Max_Yearly_Trips_Planned,%Max_Yearly_Trips_Planned,Eligible_Yearly_Trips_Planned,%Eligible_Yearly_Trips_Planned,Max_Yearly_TonKm_Planned,%Max_Yearly_TonKm_Planned,Eligible_Yearly_Ton_Km_Planned,%Eligible_Yearly_Ton_Km_Planned
150,ES<>FR,2072922.5,28191746,3.434352e+10,758312.016001,1.031304e+07,1.248557e+10,2026242.5,97.748107,741046.542045,97.723170,3.364842e+10,97.976023,1.222883e+10,97.943688
111,DE<>PL,2026515.0,27560604,2.666867e+10,828753.548108,1.127105e+07,1.091652e+10,2017532.5,99.556751,825093.024770,99.558310,2.655698e+10,99.581164,1.087098e+10,99.582856
104,DE<>IT,974817.5,13257518,1.391835e+10,321906.227246,4.377925e+06,4.573114e+09,966045.0,99.100088,319118.685862,99.134052,1.375547e+10,98.829762,4.521961e+09,98.881439
183,FR<>IT,893962.5,12157890,1.252359e+10,296937.749770,4.038353e+06,4.137681e+09,878250.0,98.242376,291933.026027,98.314555,1.227851e+10,98.043017,4.062507e+09,98.183176
98,DE<>ES,483637.5,6577470,1.147493e+10,161080.218490,2.190691e+06,3.678126e+09,481725.0,99.604559,160449.386768,99.608374,1.142225e+10,99.540906,3.661137e+09,99.538114


In [ ]:
Flux_Country['%Max_Yearly_Trips_Planned'] = Flux_Country['%Max_Yearly_Trips_Planned'].apply(lambda x: f"{x:.2f}%")
Flux_Country['%Max_Yearly_TonKm_Planned'] = Flux_Country['%Max_Yearly_TonKm_Planned'].apply(lambda x: f"{x:.2f}%")

In [ ]:
Flux_Country['%Eligible_Yearly_Trips_Planned'] = Flux_Country['%Eligible_Yearly_Trips_Planned'].apply(lambda x: f"{x:.2f}%")
Flux_Country['%Eligible_Yearly_Ton_Km_Planned'] = Flux_Country['%Eligible_Yearly_Ton_Km_Planned'].apply(lambda x: f"{x:.2f}%")

In [ ]:
import pandas as pd

# Étape 1 : Obtenez la liste de tous les pays uniques
unique_countries = set()
for corridor in Flux_Country['Corridor_Country']:
    countries = corridor.split('<>')
    unique_countries.update(countries)

# Étape 2 : Créez le DataFrame Country_Analysis avec des valeurs initiales à zéro
columns = ['Country', 'Max_Yearly_Trips_Tot','Max_Yearly_TonKm_Tot' ,'Max_Yearly_Trips_Planned',
            'Eligible_Yearly_Trips_Planned','Eligible_Yearly_Trips_Tot','Max_Yearly_TonKm_Planned','Eligible_Yearly_Ton_Km_Tot','Eligible_Yearly_Ton_Km_Planned']
Country_Analysis = pd.DataFrame(columns=columns)

# Étape 3 : Parcourez chaque pays unique et initialisez les valeurs à zéro
for country in unique_countries:
    row_data = {'Country': country,
                'Max_Yearly_Trips_Tot': 0,
                'Max_Yearly_TonKm_Tot':0,
                'Max_Yearly_Trips_Planned':0,
                'Max_Yearly_TonKm_Planned':0,
                'Eligible_Yearly_Trips_Tot': 0,
                'Eligible_Yearly_Ton_Km_Tot':0,
                'Eligible_Yearly_Trips_Planned':0,
                'Eligible_Yearly_Ton_Km_Planned':0}

    # Parcourez les corridors de Flux_Country
    for index, row in Flux_Country.iterrows():
        corridor_countries = row['Corridor_Country'].split('<>')
        if country in corridor_countries:
            # Ajoutez les valeurs correspondantes
            row_data['Max_Yearly_Trips_Tot'] += row.get('Max_Yearly_Trips_Tot', 0)
            row_data['Max_Yearly_TonKm_Tot'] += row.get('Max_Yearly_TonKm_Tot', 0)
            row_data['Max_Yearly_Trips_Planned'] += row.get('Max_Yearly_Trips_Planned', 0)
            row_data['Max_Yearly_TonKm_Planned'] += row.get('Max_Yearly_TonKm_Planned', 0)
            row_data['Eligible_Yearly_Trips_Tot'] += row.get('Eligible_Yearly_Trips_Tot', 0)
            row_data['Eligible_Yearly_Ton_Km_Tot'] += row.get('Eligible_Yearly_Ton_Km_Tot', 0)
            row_data['Eligible_Yearly_Trips_Planned'] += row.get('Eligible_Yearly_Trips_Planned', 0)
            row_data['Eligible_Yearly_Ton_Km_Planned'] += row.get('Eligible_Yearly_Ton_Km_Planned', 0)

    # Ajoutez la ligne au DataFrame Country_Analysis
    Country_Analysis = pd.concat([Country_Analysis, pd.DataFrame([row_data])], ignore_index=True)

# Tri par ordre décroissant de Max_Yearly_Trips_Tot
Country_Analysis = Country_Analysis.sort_values(by='Max_Yearly_TonKm_Tot', ascending=False)

# Réinitialisez les index
Country_Analysis.reset_index(drop=True, inplace=True)

# Affichez le résultat
Country_Analysis.head(20)


,Country,Max_Yearly_Trips_Tot,Max_Yearly_TonKm_Tot,Max_Yearly_Trips_Planned,Eligible_Yearly_Trips_Planned,Eligible_Yearly_Trips_Tot,Max_Yearly_TonKm_Planned,Eligible_Yearly_Ton_Km_Tot,Eligible_Yearly_Ton_Km_Planned
0,DE,7391832.5,1.059213e+11,7111122.5,2.652685e+06,2.749685e+06,1.013119e+11,3.892629e+10,3.736198e+10
1,FR,5679867.5,8.965110e+10,5153310.0,1.760740e+06,1.926307e+06,8.078388e+10,2.997787e+10,2.726663e+10
2,ES,4540040.0,8.670494e+10,4300615.0,1.442213e+06,1.513590e+06,8.129781e+10,2.745762e+10,2.601106e+10
3,PL,4644970.0,7.112927e+10,4353462.5,1.586230e+06,1.682573e+06,6.506234e+10,2.493867e+10,2.297384e+10
4,IT,4293165.0,6.824705e+10,4011300.0,1.274204e+06,1.351406e+06,6.300503e+10,2.087433e+10,1.961332e+10
5,NL,1983240.0,3.103361e+10,1778335.0,5.738389e+05,6.389844e+05,2.781955e+10,9.939250e+09,8.920674e+09
6,UK,1438847.5,2.815250e+10,114070.0,3.603467e+04,4.182319e+05,2.027473e+09,7.964273e+09,6.365256e+08
7,BE,1547700.0,2.330728e+10,1390657.5,3.357907e+05,3.696019e+05,2.102596e+10,5.290873e+09,4.814967e+09
8,AT,1465340.0,1.905068e+10,1349952.5,5.227535e+05,5.680220e+05,1.747497e+10,7.316988e+09,6.710182e+09
9,CZ,1259825.0,1.822952e+10,1208947.5,4.462110e+05,4.631132e+05,1.710937e+10,6.452196e+09,6.115722e+09


In [ ]:
Country_Analysis.head()

,Country,Max_Yearly_Trips_Tot,Max_Yearly_TonKm_Tot,Max_Yearly_Trips_Planned,Eligible_Yearly_Trips_Planned,Eligible_Yearly_Trips_Tot,Max_Yearly_TonKm_Planned,Eligible_Yearly_Ton_Km_Tot,Eligible_Yearly_Ton_Km_Planned
0,DE,7391832.5,1.059213e+11,7111122.5,2.652685e+06,2.749685e+06,1.013119e+11,3.892629e+10,3.736198e+10
1,FR,5679867.5,8.965110e+10,5153310.0,1.760740e+06,1.926307e+06,8.078388e+10,2.997787e+10,2.726663e+10
2,ES,4540040.0,8.670494e+10,4300615.0,1.442213e+06,1.513590e+06,8.129781e+10,2.745762e+10,2.601106e+10
3,PL,4644970.0,7.112927e+10,4353462.5,1.586230e+06,1.682573e+06,6.506234e+10,2.493867e+10,2.297384e+10
4,IT,4293165.0,6.824705e+10,4011300.0,1.274204e+06,1.351406e+06,6.300503e+10,2.087433e+10,1.961332e+10


In [ ]:
Country_Notplanned = Country_Analysis[Country_Analysis['Max_Yearly_Trips_Planned'] == 0]
Country_Notplanned.head(15)

,Country,Max_Yearly_Trips_Tot,Max_Yearly_TonKm_Tot,Max_Yearly_Trips_Planned,Eligible_Yearly_Trips_Planned,Eligible_Yearly_Trips_Tot,Max_Yearly_TonKm_Planned,Eligible_Yearly_Ton_Km_Tot,Eligible_Yearly_Ton_Km_Planned
21,RU,45732.5,2.646248e+09,0.0,0.0,14634.400000,0.0,8.467992e+08,0.0
27,IE,120535.0,1.667303e+09,0.0,0.0,35539.954541,0.0,4.595874e+08,0.0
29,KZ,13475.0,7.675774e+08,0.0,0.0,4312.000000,0.0,2.456248e+08,0.0
31,UA,15130.0,3.664350e+08,0.0,0.0,4841.600000,0.0,1.172592e+08,0.0
35,TR,7560.0,1.305327e+08,0.0,0.0,2419.200000,0.0,4.177045e+07,0.0
37,CY,2262.5,6.793051e+07,0.0,0.0,668.000000,0.0,1.871116e+07,0.0
42,SM,440.0,5.962036e+06,0.0,0.0,140.800000,0.0,1.907852e+06,0.0


In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,Step,Go_Live_Planned,Max_Yearly_Ton_Km,Category_Distance,%Type_Goods,Eligible_Yearly_Trips,Eligible_Yearly_Tons,Eligible_Yearly_Ton_Km,Distance_ECTN,Ecart_relatif_distance
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,...,S1_Ph2,2030.0,2.190316e+09,KM1000-1999,0.367110,48161.149566,654991.634095,8.040867e+08,1345.530,0.10
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,...,S1_Ph3,2030.0,9.120509e+08,KM1000-1999,0.376706,21912.968548,298016.372259,3.435747e+08,1183.137,0.03
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,...,S1_Ph2,2030.0,7.621966e+08,KM1000-1999,0.322823,18069.224319,245741.450739,2.460548e+08,1040.072,0.04
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,...,S1_Ph2,2030.0,7.538713e+08,KM1000-1999,0.367110,18290.333671,248748.537920,2.767536e+08,1170.581,0.05
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,...,S1_Ph3,2030.0,5.406422e+08,KM500-999,0.414824,20408.289453,277552.736564,2.242712e+08,856.669,0.06


In [ ]:
import re

def extract_city_and_country(corridor):
    matches = re.findall(r'Hub_([\w\s-]+)\((\w+)\)', corridor)
    cities = []
    for match in matches:
        cities.append([match[0], match[1]])
    return cities

corridor = "Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Vigo(ES)"
cities = extract_city_and_country(corridor)
print(cities)


[['Barcelona - Sant andreu de la barca', 'ES'], ['Vigo', 'ES']]


In [ ]:
import pandas as pd

# Supposons que vous ayez déjà chargé votre DataFrame "Flux_Scaled"

# Définissez une fonction personnalisée pour détecter les erreurs de boucle
def detect_loop_error(row):
    # Obtenez les valeurs de la colonne "Corridor_Générique" et "Routing_Final" pour la ligne actuelle
    corridor_generic = row['Corridor_Générique']
    routing_final = row['Routing_Final']

    # Obtenez les noms des villes à partir de "Corridor_Générique"
    hub_ville_o = extract_city_and_country(corridor_generic)[0][0]
    hub_ville_d = extract_city_and_country(corridor_generic)[1][0]

    # Obtenez les noms des villes à partir de "Routing_Final"
    routing_cities = routing_final.split('<>')
    first_city = routing_cities[0]
    last_city = routing_cities[-1]

    # Vérifiez les conditions d'erreur de boucle
    if first_city != hub_ville_o and first_city != hub_ville_d:
        return 'Erreur'
    elif last_city != hub_ville_o and last_city != hub_ville_d:
        return 'Erreur'
    else:
        return 'Aucune erreur'

# Appliquez la fonction pour créer la colonne "Erreur"
Flux_Scaled['Erreur'] = Flux_Scaled.apply(detect_loop_error, axis=1)

Test = Flux_Scaled[Flux_Scaled['Erreur']=="Erreur"]
Test

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,Go_Live_Planned,Max_Yearly_Ton_Km,Category_Distance,%Type_Goods,Eligible_Yearly_Trips,Eligible_Yearly_Tons,Eligible_Yearly_Ton_Km,Distance_ECTN,Ecart_relatif_distance,Erreur
78,Hub_Aarhus(DK) <> Hub_Oslo(NO),329664,24240.0,858.6498,116479.0,250487.0,"[2500924, 2610062, 2610093, 2610094, 1036108, ...","[130103, 124372, 130103, 262750, 262751, 26275...",Aarhus<>Oslo,900.764,...,Erreur,2.969495e+08,KM500-999,0.299956,7270.924308,98884.570591,8.907166e+07,0.0,-1.0,Erreur


In [ ]:
Flux_Scaled_erreurs = Flux_Scaled[Flux_Scaled['Erreur'] == 'Erreur : Boucle']
Flux_Scaled_erreurs

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,Go_Live_Planned,Max_Yearly_Ton_Km,Category_Distance,%Type_Goods,Eligible_Yearly_Trips,Eligible_Yearly_Tons,Eligible_Yearly_Ton_Km,Distance_ECTN,Ecart_relatif_distance,Erreur


In [ ]:
# Créez la colonne Nb_Trucks_Electric_LA
Flux_Scaled['Nb_Trucks_Electric_LA'] = (Flux_Scaled['<200'] + Flux_Scaled['[200,300]']) * Flux_Scaled['Eligible_Yearly_Trips'] / (4*240)

# Créez la colonne Nb_Trucks_Electric_HA
Flux_Scaled['Nb_Trucks_Electric_HA'] = Flux_Scaled['[300,350]'] * Flux_Scaled['Eligible_Yearly_Trips'] / (4*240)

# Créez la colonne Nb_Trucks_BioGaz
Flux_Scaled['Nb_Trucks_BioGaz'] = Flux_Scaled['>350'] * Flux_Scaled['Eligible_Yearly_Trips'] / (4*240)


In [ ]:
Flux_Scaled.head()

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,%Type_Goods,Eligible_Yearly_Trips,Eligible_Yearly_Tons,Eligible_Yearly_Ton_Km,Distance_ECTN,Ecart_relatif_distance,Erreur,Nb_Trucks_Electric_LA,Nb_Trucks_Electric_HA,Nb_Trucks_BioGaz
832,Hub_Barcelona - Sant andreu de la barca(ES) <>...,1784184,131190.0,1275.775538,108641.0,195348.0,"[1306531, 1001666, 1306683, 1001660, 1000200, ...","[101480, 195360, 101480, 101477, 101477, 19536...",Barcelona - Sant andreu de la barca<>Montpelli...,1227.629,...,0.367110,48161.149566,654991.634095,8.040867e+08,1345.530,0.10,Aucune erreur,150.503592,100.335728,0.0
869,Hub_Barcelona - Sant andreu de la barca(ES) <>...,791112,58170.0,1166.914750,105995.0,195348.0,"[1305931, 1305622, 1305629, 2500017, 2500018, ...","[106000, 190533, 106000, 190531, 190530, 19053...",Porto - Lagoa<>Benavente<>Valladolid<>Miranda ...,1152.872,...,0.376706,21912.968548,298016.372259,3.435747e+08,1183.137,0.03,Aucune erreur,22.826009,68.478027,0.0
4434,Hub_Lille(FR) <> Hub_Milan - assago(IT),761226,55972.5,1022.996765,104770.0,108641.0,"[1006101, 1038077, 1006080, 1006114, 1014966, ...","[104756, 104784, 104756, 125247, 104755, 12524...",Milan - Monza<>Milan - assago<>Lucerne<>Mulhou...,1001.275,...,0.322823,18069.224319,245741.450739,2.460548e+08,1040.072,0.04,Aucune erreur,56.466326,18.822109,0.0
4448,Hub_Lille(FR) <> Hub_Pampelune(ES),677586,49822.5,1160.321000,102027.0,108641.0,"[1000921, 1001425, 1000934, 1002024, 1000904, ...","[101001, 193590, 101701, 101001, 100984, 10170...",Pampelune<>Bayonne<>Bordeaux<>Poitiers<>Le Man...,1112.584,...,0.367110,18290.333671,248748.537920,2.767536e+08,1170.581,0.05,Aucune erreur,38.104862,38.104862,0.0
4532,Hub_Linz(AT) <> Hub_Moers(DE),669086,49197.5,845.661628,123924.0,196493.0,"[1300320, 1300312, 1018213, 1300314, 1300315, ...","[196519, 112533, 196513, 196519, 196513, 11258...",Moers<>Wuppertal<>Cologne<>Francfort - Mayence...,808.031,...,0.414824,20408.289453,277552.736564,2.242712e+08,856.669,0.06,Aucune erreur,42.517270,21.258635,0.0


In [ ]:
import pandas as pd

# Assurez-vous d'avoir chargé les DataFrames Flux_Scaled et Terminaux_Data

# Chemin du fichier Excel de sortie
nom_du_fichier_excel = "/content/drive/My Drive/ECTN_Routing/Flux.xlsx"

# Créez un objet ExcelWriter
with pd.ExcelWriter(nom_du_fichier_excel, engine='xlsxwriter') as writer:
    # Enregistrez Flux_Scaled dans la feuille Sheet1
    Flux_Scaled.to_excel(writer, sheet_name='Corridor_Integrated', index=False)

    # Enregistrez Terminaux_Data dans la feuille Sheet2
    Terminaux_Data.to_excel(writer, sheet_name='Terminaux_Integrated', index=False)
    # Enregistrez Flux_Country dans la feuille Sheet3
    Flux_Country.to_excel(writer, sheet_name='Country<>Country', index=False)
    # Enregistrez Country_Analysis dans la feuille Sheet4
    Country_Analysis.to_excel(writer, sheet_name='Country', index=False)
    # Enregistrez  dans la feuille Sheet5
    Flux_Data_Générique_Full.to_excel(writer, sheet_name='Corridor_Tot', index=False)
    Flux_Data.to_excel(writer, sheet_name='Correspondances', index=False)

# Exemple Routing Map

In [ ]:
Nodes_Data_Original =pd.read_csv("drive/MyDrive/ECTN_Routing/03_network-nodes.csv")

In [ ]:
Nodes_Data_Original.columns

Index(['Unnamed: 0', 'Network_Node_ID', 'Network_Node_X', 'Network_Node_Y',
       'ETISplus_Zone_ID', 'Country'],
      dtype='object')

In [ ]:
# Assurez-vous que les noms des colonnes correspondent à ceux de votre DataFrame
Nodes_Data_Original['Geometric_Center'] = Nodes_Data_Original.apply(lambda row: [row['Network_Node_Y'], row[ 'Network_Node_X']], axis=1)


In [ ]:
# Extraction de la ligne correspondante
ligne_extraite = Flux_Scaled.loc[Flux_Scaled['Corridor_Générique'] == 'Hub_Barcelona - Sant andreu de la barca(ES) <> Hub_Brescia(IT)']
ligne_extraite

,Corridor_Générique,Max_Yearly_Tons,Max_Yearly_Trips,Distance_Direct_ETIS_plus,ID_Node_Scaled_Origin,ID_Node_Scaled_Destination,Edge_Path,Nodes_Path,Routing,Distance_Direct,...,%Type_Goods,Eligible_Yearly_Trips,Eligible_Yearly_Tons,Eligible_Yearly_Ton_Km,Distance_ECTN,Ecart_relatif_distance,Erreur,Nb_Trucks_Electric_LA,Nb_Trucks_Electric_HA,Nb_Trucks_BioGaz
788,Hub_Barcelona - Sant andreu de la barca(ES) <>...,240958,17717.5,1071.885,104823.0,195348.0,"[1006172, 1006222, 1006491, 1006494, 1006207, ...","[104824, 104823, 104856, 104824, 104861, 10485...",Parme<>Brescia<>Milan - assago<>Gênes<>Nice<>M...,1052.825,...,0.322823,5719.620919,77786.844494,8.189593e+07,1099.622,0.04,Aucune erreur,11.915877,11.915877,0.0


In [ ]:
ligne_extraite['Nodes_Path'].apply(lambda path_list: [int(x) for x in path_list])

788    [104824, 104823, 104856, 104824, 104861, 10485...
Name: Nodes_Path, dtype: object

In [ ]:
List_nodes = ligne_extraite['Nodes_Path']

In [ ]:
List_nodes =List_nodes[770]

KeyError: ignored

In [ ]:
Nodes_Data.columns

In [ ]:
Center = []

In [ ]:
for node_id in List_nodes:
    # Recherchez l'entier dans la colonne Network_Node_ID de Nodes_Data
    row = Nodes_Data_Original.loc[Nodes_Data_Original['Network_Node_ID'] == node_id]

    # Si une correspondance est trouvée, ajoutez la valeur de Geometric_Center à Geom_Center
    if not row.empty:
        Center.append(row['Geometric_Center'].values[0])
    else:
        # Si aucune correspondance n'est trouvée, ajoutez une valeur vide à Geom_Center
        Center.append(None)



In [ ]:
import folium
import folium
from folium.plugins import BeautifyIcon
import math

# Création de la carte centrée sur la France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Tracer la route en respectant l'ordre de passage
for i in range(len(Center)-1):
    start_point = Center[i]
    end_point = Center[i+1]
    folium.PolyLine([start_point, end_point], color="blue", weight=2.5, opacity=1).add_to(m)

# Ajouter des marqueurs numérotés pour indiquer l'ordre de passage
for i, point in enumerate(Center):
    folium.Marker(point, icon=folium.Icon(color='red'), tooltip=f"Point {i+1}").add_to(m)

# Afficher la carte
m.save('route.html')  # Sauvegarder la carte au format HTML
m


In [ ]:
import pandas as pd

# Supposons que vous avez déjà un DataFrame avec les colonnes "Routing" et "Distance"
# Exemple de données
data = {'Routing': ['Paris<>Orléans<>Tours<>Bordeaux<>Gijon', 'Barcelone<>Séville<>Valence<>Saragosse', 'Milan<>Turin<>Rome<>Naples<>Lille<>Londres'],
        'Distance': [[100, 200, 150, 200], [100, 360, 200], [100, 50, 50, 50, 300]]}

df = pd.DataFrame(data)

# Fonction pour obtenir le "Routing_Réduit"
def get_reduced_routing(row):
    routing = row['Routing'].split('<>')
    distances = row['Distance']
    reduced_routing = [routing[0]]
    current_distance = 0

    for i in range(len(routing) - 1):
        current_distance += distances[i]
        if current_distance >= 250 and current_distance <= 350:
            reduced_routing.append(routing[i])
            current_distance = 0
        elif current_distance > 350:
            return "Erreur"

    # Ajoutez la dernière étape
    reduced_routing.append(routing[-1])

    return '<>'.join(reduced_routing)

# Appliquer la fonction pour créer la colonne "Routing_Réduit"
df['Routing_Réduit'] = df.apply(get_reduced_routing, axis=1)

# Afficher le DataFrame final
print(df)
